In [1]:
from sympy import *
from sympy.physics.quantum import *
import pandas as pd
from scipy.sparse import csc_matrix

In [2]:
n={}
n[0]=Matrix([0,0])
n[1]=Matrix([cos(rad(-120)),sin(rad(-120))])
n[2]=Matrix([cos(rad(-60)),sin(rad(-60))])

In [3]:
b={}
b[1]=Matrix([cos(rad(30)),sin(rad(30))])
b[2]=Matrix([cos(rad(-30)),sin(rad(-30))])
K={}
K['+']=(b[1]+b[2])/3
K['-']=-(b[1]+b[2])/3

In [4]:
b_idx={}
b_idx[1]=Matrix([1,0])
b_idx[2]=Matrix([0,1])
K_idx={}
K_idx['+']=(b_idx[1]+b_idx[2])/3
K_idx['-']=-(b_idx[1]+b_idx[2])/3

In [5]:
[exp(I*2*4*pi/sqrt(3)*K['+'].dot(n[i])) for i in range(3)]

[1, exp(2*I*pi/3), exp(-2*I*pi/3)]

In [6]:
[exp(-I*2*4*pi/sqrt(3)*K['+'].dot(n[i])) for i in range(3)]

[1, exp(-2*I*pi/3), exp(2*I*pi/3)]

In [7]:
theta={i:symbols(rf'\theta_{{κ_{i}}}',real=True) for i in range(1,5)}

In [115]:
# Gauge 2
# W={(valley,i):1/sqrt(2)*Matrix([[exp(-I * theta[i]/2), exp(I * theta[i]/2)],[exp(-I * theta[i]/2),-exp(I * theta[i]/2)]]) for i in range(1,5) for valley in '+-'}


W={('+',i):1/sqrt(2)*Matrix([[1,exp(I*theta[i])],[1,-exp(I*theta[i])]]) for i in range(1,5)}
W.update({('-',i):1/sqrt(2)*Matrix([[-exp(I*theta[i]),1],[exp(I*theta[i]),1]]) for i in range(1,5)}) # Gauge 1
# W.update({('-',i):1/sqrt(2)*Matrix([[1,-exp(-I*theta[i])],[1,exp(-I*theta[i])]]) for i in range(1,5)}) # Gauge 3


In [116]:
sigma={}
sigma['0']=Matrix([[1,0],[0,1]])
sigma['x']=Matrix([[0,1],[1,0]])
sigma['y']=Matrix([[0,-I],[I,0]])
sigma['z']=Matrix([[1,0],[0,-1]])
sigma['+']=Matrix([[0,1],[0,0]])
sigma['-']=Matrix([[0,0],[1,0]])
sigma['u']=Matrix([[1,0],[0,0]])
sigma['d']=Matrix([[0,0],[0,1]])

In [117]:
spin=lambda σ: '↑' if σ =='u' else '↓'

In [118]:
c={"".join([λ,l,σ,κ]):Operator(f'c_{{{λ}K,{l},{spin(σ)},κ_{κ}}}') for λ in '+-' for l in '+-AB' for σ in 'ud' for κ in '1234'}
psi={"".join([λ,l,σ,κ]):Operator(f'\psi_{{{λ}K,{l},{spin(σ)},κ_{κ}}}') for λ in '+-' for l in '+-AB' for σ in 'ud' for κ in '1234'}

In [119]:
c_vec={κ:Matrix([c["".join([λ,l,σ,str(κ)])] for σ in 'ud' for λ in '+-' for l in 'AB']) for κ in range(1,5)}
c_d_vec={κ:Matrix([Dagger(c["".join([λ,l,σ,str(κ)])]) for σ in 'ud' for λ in '+-' for l in 'AB']) for κ in range(1,5)}
cl_vec={κ:Matrix([c["".join([λ,l,σ,str(κ)])] for σ in 'ud' for λ in '+-' for l in '+-']) for κ in range(1,5)}
cl_d_vec={κ:Matrix([Dagger(c["".join([λ,l,σ,str(κ)])]) for σ in 'ud' for λ in '+-' for l in '+-']) for κ in range(1,5)}

In [120]:
psi_vec={κ:Matrix([psi["".join([λ,l,σ,str(κ)])] for σ in 'ud' for λ in '+-' for l in 'AB']) for κ in range(1,5)}
psi_d_vec={κ:Matrix([Dagger(psi["".join([λ,l,σ,str(κ)])]) for σ in 'ud' for λ in '+-' for l in 'AB']) for κ in range(1,5)}
psil_vec={κ:Matrix([psi["".join([λ,l,σ,str(κ)])] for σ in 'ud' for λ in '+-' for l in '+-']) for κ in range(1,5)}
psil_d_vec={κ:Matrix([Dagger(psi["".join([λ,l,σ,str(κ)])]) for σ in 'ud' for λ in '+-' for l in '+-']) for κ in range(1,5)}

In [121]:
M=kronecker_product(sigma['0'],kronecker_product(sigma['u'],sigma['0'])+kronecker_product(sigma['d'],I*sigma['y']))

In [122]:
# WW={i:kronecker_product(sigma['0'],sigma['0'],(W[i])) for i in range(1,5)}
WW={i:kronecker_product(kronecker_product(sigma['0'],sigma['u']),W[('+',i)]) + kronecker_product(kronecker_product(sigma['0'],sigma['d']),W[('-',i)]) for i in range(1,5)}

In [123]:
WW[1]

Matrix([
[sqrt(2)/2,  sqrt(2)*exp(I*\theta_{κ_1})/2,                              0,         0,         0,                              0,                              0,         0],
[sqrt(2)/2, -sqrt(2)*exp(I*\theta_{κ_1})/2,                              0,         0,         0,                              0,                              0,         0],
[        0,                              0, -sqrt(2)*exp(I*\theta_{κ_1})/2, sqrt(2)/2,         0,                              0,                              0,         0],
[        0,                              0,  sqrt(2)*exp(I*\theta_{κ_1})/2, sqrt(2)/2,         0,                              0,                              0,         0],
[        0,                              0,                              0,         0, sqrt(2)/2,  sqrt(2)*exp(I*\theta_{κ_1})/2,                              0,         0],
[        0,                              0,                              0,         0, sqrt(2)/2, -sqrt(2)*exp(I*\theta_{

In [124]:
rules_c2cl=sum([list(zip(c_vec[i],WW[i].T*cl_vec[i])) for i in range(1,5)],[])

In [125]:
rules_c2psi=sum([list(zip(c_vec[i],M.T*psi_vec[i])) for i in range(1,5)],[])
rules_psi2c=sum([list(zip(psi_vec[i],M*c_vec[i])) for i in range(1,5)],[])
rules_psil2psi=sum([list(zip(psil_vec[i],conjugate(WW[i])*psi_vec[i])) for i in range(1,5)],[])
rules_psi2psil=sum([list(zip(psi_vec[i],(WW[i].T)*psil_vec[i])) for i in range(1,5)],[])

In [126]:
c_vec[1]

Matrix([
[c_{+K,A,↑,κ_1}],
[c_{+K,B,↑,κ_1}],
[c_{-K,A,↑,κ_1}],
[c_{-K,B,↑,κ_1}],
[c_{+K,A,↓,κ_1}],
[c_{+K,B,↓,κ_1}],
[c_{-K,A,↓,κ_1}],
[c_{-K,B,↓,κ_1}]])

In [127]:
c_vec[1].subs(rules_c2cl)

Matrix([
[                                         sqrt(2)*c_{+K,+,↑,κ_1}/2 + sqrt(2)*c_{+K,-,↑,κ_1}/2],
[ sqrt(2)*exp(I*\theta_{κ_1})*c_{+K,+,↑,κ_1}/2 - sqrt(2)*exp(I*\theta_{κ_1})*c_{+K,-,↑,κ_1}/2],
[-sqrt(2)*exp(I*\theta_{κ_1})*c_{-K,+,↑,κ_1}/2 + sqrt(2)*exp(I*\theta_{κ_1})*c_{-K,-,↑,κ_1}/2],
[                                         sqrt(2)*c_{-K,+,↑,κ_1}/2 + sqrt(2)*c_{-K,-,↑,κ_1}/2],
[                                         sqrt(2)*c_{+K,+,↓,κ_1}/2 + sqrt(2)*c_{+K,-,↓,κ_1}/2],
[ sqrt(2)*exp(I*\theta_{κ_1})*c_{+K,+,↓,κ_1}/2 - sqrt(2)*exp(I*\theta_{κ_1})*c_{+K,-,↓,κ_1}/2],
[-sqrt(2)*exp(I*\theta_{κ_1})*c_{-K,+,↓,κ_1}/2 + sqrt(2)*exp(I*\theta_{κ_1})*c_{-K,-,↓,κ_1}/2],
[                                         sqrt(2)*c_{-K,+,↓,κ_1}/2 + sqrt(2)*c_{-K,-,↓,κ_1}/2]])

# $H_0$

In [128]:
H0=(Matrix([Dagger(c['+Au1']),Dagger(c['+Bu1'])]).T * (sigma['x']*cos(theta[1])+sigma['y']*sin(theta[1])) * Matrix([c['+Au1'],c['+Bu1']]) + Matrix([Dagger(c['-Au1']),Dagger(c['-Bu1'])]).T * conjugate(-sigma['x']*cos(theta[1])-sigma['y']*sin(theta[1])) * Matrix([c['-Au1'],c['-Bu1']]))[0,0].rewrite(exp)

In [25]:
H0

exp(I*\theta_{κ_1})*Dagger(c_{+K,B,↑,κ_1})*c_{+K,A,↑,κ_1} - exp(I*\theta_{κ_1})*Dagger(c_{-K,A,↑,κ_1})*c_{-K,B,↑,κ_1} + exp(-I*\theta_{κ_1})*Dagger(c_{+K,A,↑,κ_1})*c_{+K,B,↑,κ_1} - exp(-I*\theta_{κ_1})*Dagger(c_{-K,B,↑,κ_1})*c_{-K,A,↑,κ_1}

In [26]:
H0.subs(rules_c2cl).expand()

Dagger(c_{+K,+,↑,κ_1})*c_{+K,+,↑,κ_1} - Dagger(c_{+K,-,↑,κ_1})*c_{+K,-,↑,κ_1} + Dagger(c_{-K,+,↑,κ_1})*c_{-K,+,↑,κ_1} - Dagger(c_{-K,-,↑,κ_1})*c_{-K,-,↑,κ_1}

In [27]:
H0.subs(rules_c2psi)

exp(I*\theta_{κ_1})*Dagger(\psi_{+K,B,↑,κ_1})*\psi_{+K,A,↑,κ_1} + exp(I*\theta_{κ_1})*Dagger(\psi_{-K,B,↑,κ_1})*\psi_{-K,A,↑,κ_1} + exp(-I*\theta_{κ_1})*Dagger(\psi_{+K,A,↑,κ_1})*\psi_{+K,B,↑,κ_1} + exp(-I*\theta_{κ_1})*Dagger(\psi_{-K,A,↑,κ_1})*\psi_{-K,B,↑,κ_1}

In [28]:
H0_psi=(Matrix(psi_d_vec[1][:4]).T * kronecker_product(sigma['0'],(sigma['x']*cos(theta[1])+sigma['y']*sin(theta[1]))).rewrite(exp) * Matrix(psi_vec[1][:4]))[0,0]

In [29]:
(H0_psi-H0.subs(rules_c2psi))

0

In [30]:
psi_vec[1].subs(rules_psi2psil)

Matrix([
[                                         sqrt(2)*\psi_{+K,+,↑,κ_1}/2 + sqrt(2)*\psi_{+K,-,↑,κ_1}/2],
[ sqrt(2)*exp(I*\theta_{κ_1})*\psi_{+K,+,↑,κ_1}/2 - sqrt(2)*exp(I*\theta_{κ_1})*\psi_{+K,-,↑,κ_1}/2],
[-sqrt(2)*exp(I*\theta_{κ_1})*\psi_{-K,+,↑,κ_1}/2 + sqrt(2)*exp(I*\theta_{κ_1})*\psi_{-K,-,↑,κ_1}/2],
[                                         sqrt(2)*\psi_{-K,+,↑,κ_1}/2 + sqrt(2)*\psi_{-K,-,↑,κ_1}/2],
[                                         sqrt(2)*\psi_{+K,+,↓,κ_1}/2 + sqrt(2)*\psi_{+K,-,↓,κ_1}/2],
[ sqrt(2)*exp(I*\theta_{κ_1})*\psi_{+K,+,↓,κ_1}/2 - sqrt(2)*exp(I*\theta_{κ_1})*\psi_{+K,-,↓,κ_1}/2],
[-sqrt(2)*exp(I*\theta_{κ_1})*\psi_{-K,+,↓,κ_1}/2 + sqrt(2)*exp(I*\theta_{κ_1})*\psi_{-K,-,↓,κ_1}/2],
[                                         sqrt(2)*\psi_{-K,+,↓,κ_1}/2 + sqrt(2)*\psi_{-K,-,↓,κ_1}/2]])

In [31]:
psi_d_vec[1].subs(rules_psi2psil)

Matrix([
[                                           sqrt(2)*Dagger(\psi_{+K,+,↑,κ_1})/2 + sqrt(2)*Dagger(\psi_{+K,-,↑,κ_1})/2],
[ sqrt(2)*exp(-I*\theta_{κ_1})*Dagger(\psi_{+K,+,↑,κ_1})/2 - sqrt(2)*exp(-I*\theta_{κ_1})*Dagger(\psi_{+K,-,↑,κ_1})/2],
[-sqrt(2)*exp(-I*\theta_{κ_1})*Dagger(\psi_{-K,+,↑,κ_1})/2 + sqrt(2)*exp(-I*\theta_{κ_1})*Dagger(\psi_{-K,-,↑,κ_1})/2],
[                                           sqrt(2)*Dagger(\psi_{-K,+,↑,κ_1})/2 + sqrt(2)*Dagger(\psi_{-K,-,↑,κ_1})/2],
[                                           sqrt(2)*Dagger(\psi_{+K,+,↓,κ_1})/2 + sqrt(2)*Dagger(\psi_{+K,-,↓,κ_1})/2],
[ sqrt(2)*exp(-I*\theta_{κ_1})*Dagger(\psi_{+K,+,↓,κ_1})/2 - sqrt(2)*exp(-I*\theta_{κ_1})*Dagger(\psi_{+K,-,↓,κ_1})/2],
[-sqrt(2)*exp(-I*\theta_{κ_1})*Dagger(\psi_{-K,+,↓,κ_1})/2 + sqrt(2)*exp(-I*\theta_{κ_1})*Dagger(\psi_{-K,-,↓,κ_1})/2],
[                                           sqrt(2)*Dagger(\psi_{-K,+,↓,κ_1})/2 + sqrt(2)*Dagger(\psi_{-K,-,↓,κ_1})/2]])

In [32]:
H0_psi.subs(rules_psi2psil).expand()

-exp(2*I*\theta_{κ_1})*Dagger(\psi_{-K,+,↑,κ_1})*\psi_{-K,+,↑,κ_1}/2 + exp(2*I*\theta_{κ_1})*Dagger(\psi_{-K,+,↑,κ_1})*\psi_{-K,-,↑,κ_1}/2 - exp(2*I*\theta_{κ_1})*Dagger(\psi_{-K,-,↑,κ_1})*\psi_{-K,+,↑,κ_1}/2 + exp(2*I*\theta_{κ_1})*Dagger(\psi_{-K,-,↑,κ_1})*\psi_{-K,-,↑,κ_1}/2 + Dagger(\psi_{+K,+,↑,κ_1})*\psi_{+K,+,↑,κ_1} - Dagger(\psi_{+K,-,↑,κ_1})*\psi_{+K,-,↑,κ_1} - exp(-2*I*\theta_{κ_1})*Dagger(\psi_{-K,+,↑,κ_1})*\psi_{-K,+,↑,κ_1}/2 - exp(-2*I*\theta_{κ_1})*Dagger(\psi_{-K,+,↑,κ_1})*\psi_{-K,-,↑,κ_1}/2 + exp(-2*I*\theta_{κ_1})*Dagger(\psi_{-K,-,↑,κ_1})*\psi_{-K,+,↑,κ_1}/2 + exp(-2*I*\theta_{κ_1})*Dagger(\psi_{-K,-,↑,κ_1})*\psi_{-K,-,↑,κ_1}/2

In [33]:
vF=Symbol('v_F',real=True)
B=Symbol('B',real=True)

In [34]:
H_0=(psi_d_vec[1].T * (kronecker_product(sigma['0'], sigma['0'], vF*(sigma['x']*cos(theta[1])+sigma['y']*sin(theta[1]))) + B*kronecker_product(sigma['z'],sigma['0'],sigma['0'])) * psi_vec[1])[0,0]

In [35]:
H_0.subs(rules_psi2psil).rewrite(exp).expand().simplify()

exp(-2*I*\theta_{κ_1})*(-v_F*Dagger(\psi_{-K,+,↑,κ_1})*\psi_{-K,+,↑,κ_1} - v_F*Dagger(\psi_{-K,+,↑,κ_1})*\psi_{-K,-,↑,κ_1} - v_F*Dagger(\psi_{-K,+,↓,κ_1})*\psi_{-K,+,↓,κ_1} - v_F*Dagger(\psi_{-K,+,↓,κ_1})*\psi_{-K,-,↓,κ_1} + v_F*Dagger(\psi_{-K,-,↑,κ_1})*\psi_{-K,+,↑,κ_1} + v_F*Dagger(\psi_{-K,-,↑,κ_1})*\psi_{-K,-,↑,κ_1} + v_F*Dagger(\psi_{-K,-,↓,κ_1})*\psi_{-K,+,↓,κ_1} + v_F*Dagger(\psi_{-K,-,↓,κ_1})*\psi_{-K,-,↓,κ_1} + exp(2*I*\theta_{κ_1})*(2*B*Dagger(\psi_{+K,+,↑,κ_1})*\psi_{+K,+,↑,κ_1} - 2*B*Dagger(\psi_{+K,+,↓,κ_1})*\psi_{+K,+,↓,κ_1} + 2*B*Dagger(\psi_{+K,-,↑,κ_1})*\psi_{+K,-,↑,κ_1} - 2*B*Dagger(\psi_{+K,-,↓,κ_1})*\psi_{+K,-,↓,κ_1} + 2*B*Dagger(\psi_{-K,+,↑,κ_1})*\psi_{-K,+,↑,κ_1} - 2*B*Dagger(\psi_{-K,+,↓,κ_1})*\psi_{-K,+,↓,κ_1} + 2*B*Dagger(\psi_{-K,-,↑,κ_1})*\psi_{-K,-,↑,κ_1} - 2*B*Dagger(\psi_{-K,-,↓,κ_1})*\psi_{-K,-,↓,κ_1} - v_F*exp(2*I*\theta_{κ_1})*Dagger(\psi_{-K,+,↑,κ_1})*\psi_{-K,+,↑,κ_1} + v_F*exp(2*I*\theta_{κ_1})*Dagger(\psi_{-K,+,↑,κ_1})*\psi_{-K,-,↑,κ_1} - v_F*exp(

# $H_{10}$

In [20]:
valley_pairing=[(k1,k2,k3,k4) for k1 in '-+' for k2 in '-+' for k3 in '-+' for k4 in '-+' if (K_idx[k1]-K_idx[k2]+K_idx[k3]-K_idx[k4])%1 == Matrix([0,0])]

In [21]:
valley_pairing

[('-', '-', '-', '-'),
 ('-', '-', '+', '+'),
 ('-', '+', '+', '-'),
 ('+', '-', '-', '+'),
 ('+', '+', '-', '-'),
 ('+', '+', '+', '+')]

In [22]:
H10=sum([Dagger(c["".join([k1,l,'u','1'])]) * c["".join([k2,l,'u','2'])] * Dagger(c["".join([k3,l,'d','3'])]) * c["".join([k4,l,'d','4'])] for k1,k2,k3,k4 in valley_pairing for l in 'AB'])

1. keep only  $-\uparrow$ and $+\downarrow$
2. expand Wick
3. substitute order parameter, with inter & intra

In [110]:
# rules_proj=[(psi["".join([idx,k])],0) for k in '1234' for idx in ['++u','+-d','--u','-+d']]
# rules_proj=[(psi["".join([λ,s,k])],0) for λ in '+-' for k in '1234' for s in ['+u','-d']]
rules_proj=[(c["".join([λ,s,k])],0) for λ in '+-' for k in '1234' for s in ['+u','-d']]
rules_proj_3=[(c["".join([λ,s,k])],0) for λ in '+-' for k in '1234' for s in ['+u','-d','+d']]
rules_proj_1=[(c["".join([λ,s,k])],0) for λ in '+-' for k in '1234' for s in ['+u']]

In [25]:
# H10_pm=H10.subs(rules_c2psi).subs(rules_psi2psil).expand()
H10_pm=H10.subs(rules_c2cl).expand()

In [26]:
H10_pm

exp(I*\theta_{κ_2})*exp(-I*\theta_{κ_3})*Dagger(c_{+K,+,↑,κ_1})*c_{-K,+,↑,κ_2}*Dagger(c_{-K,+,↓,κ_3})*c_{+K,+,↓,κ_4}/4 + exp(I*\theta_{κ_2})*exp(-I*\theta_{κ_3})*Dagger(c_{+K,+,↑,κ_1})*c_{-K,+,↑,κ_2}*Dagger(c_{-K,+,↓,κ_3})*c_{+K,-,↓,κ_4}/4 - exp(I*\theta_{κ_2})*exp(-I*\theta_{κ_3})*Dagger(c_{+K,+,↑,κ_1})*c_{-K,+,↑,κ_2}*Dagger(c_{-K,-,↓,κ_3})*c_{+K,+,↓,κ_4}/4 - exp(I*\theta_{κ_2})*exp(-I*\theta_{κ_3})*Dagger(c_{+K,+,↑,κ_1})*c_{-K,+,↑,κ_2}*Dagger(c_{-K,-,↓,κ_3})*c_{+K,-,↓,κ_4}/4 - exp(I*\theta_{κ_2})*exp(-I*\theta_{κ_3})*Dagger(c_{+K,+,↑,κ_1})*c_{-K,-,↑,κ_2}*Dagger(c_{-K,+,↓,κ_3})*c_{+K,+,↓,κ_4}/4 - exp(I*\theta_{κ_2})*exp(-I*\theta_{κ_3})*Dagger(c_{+K,+,↑,κ_1})*c_{-K,-,↑,κ_2}*Dagger(c_{-K,+,↓,κ_3})*c_{+K,-,↓,κ_4}/4 + exp(I*\theta_{κ_2})*exp(-I*\theta_{κ_3})*Dagger(c_{+K,+,↑,κ_1})*c_{-K,-,↑,κ_2}*Dagger(c_{-K,-,↓,κ_3})*c_{+K,+,↓,κ_4}/4 + exp(I*\theta_{κ_2})*exp(-I*\theta_{κ_3})*Dagger(c_{+K,+,↑,κ_1})*c_{-K,-,↑,κ_2}*Dagger(c_{-K,-,↓,κ_3})*c_{+K,-,↓,κ_4}/4 + exp(I*\theta_{κ_2})*exp(-I*\thet

In [47]:
# H10_pm_proj=H10_pm.subs(rules_proj)
H10_pm_proj=H10_pm.subs(rules_proj_1)

In [48]:
H10_pm_proj.powsimp()

exp(I*(-\theta_{κ_1} + \theta_{κ_2}))*Dagger(c_{+K,-,↑,κ_1})*c_{+K,-,↑,κ_2}*Dagger(c_{-K,+,↓,κ_3})*c_{-K,+,↓,κ_4}/4 + exp(I*(-\theta_{κ_1} + \theta_{κ_2}))*Dagger(c_{+K,-,↑,κ_1})*c_{+K,-,↑,κ_2}*Dagger(c_{-K,+,↓,κ_3})*c_{-K,-,↓,κ_4}/4 + exp(I*(-\theta_{κ_1} + \theta_{κ_2}))*Dagger(c_{+K,-,↑,κ_1})*c_{+K,-,↑,κ_2}*Dagger(c_{-K,-,↓,κ_3})*c_{-K,+,↓,κ_4}/4 + exp(I*(-\theta_{κ_1} + \theta_{κ_2}))*Dagger(c_{+K,-,↑,κ_1})*c_{+K,-,↑,κ_2}*Dagger(c_{-K,-,↓,κ_3})*c_{-K,-,↓,κ_4}/4 + exp(I*(-\theta_{κ_1} + \theta_{κ_2}))*Dagger(c_{-K,-,↑,κ_1})*c_{-K,-,↑,κ_2}*Dagger(c_{+K,+,↓,κ_3})*c_{+K,+,↓,κ_4}/4 + exp(I*(-\theta_{κ_1} + \theta_{κ_2}))*Dagger(c_{-K,-,↑,κ_1})*c_{-K,-,↑,κ_2}*Dagger(c_{+K,+,↓,κ_3})*c_{+K,-,↓,κ_4}/4 + exp(I*(-\theta_{κ_1} + \theta_{κ_2}))*Dagger(c_{-K,-,↑,κ_1})*c_{-K,-,↑,κ_2}*Dagger(c_{+K,-,↓,κ_3})*c_{+K,+,↓,κ_4}/4 + exp(I*(-\theta_{κ_1} + \theta_{κ_2}))*Dagger(c_{-K,-,↑,κ_1})*c_{-K,-,↑,κ_2}*Dagger(c_{+K,-,↓,κ_3})*c_{+K,-,↓,κ_4}/4 - exp(I*(-\theta_{κ_1} + \theta_{κ_4}))*Dagger(c_{+K,-,↑,κ

In [45]:
expval={}
expval={(i,j): Symbol(rf'\langle {i}^\dagger {j} \rangle') for i in sum([list(psil_vec[i]) for i in range(1,5)],[]) for j in sum([list(psil_vec[i]) for i in range(1,5)],[])}
expval.update({(i,j): Symbol(rf'\langle {i}^\dagger {j} \rangle') for i in sum([list(cl_vec[i]) for i in range(1,5)],[]) for j in sum([list(cl_vec[i]) for i in range(1,5)],[])})

In [46]:
import re
def _idx(op):
    op_str=str(op)
    match = re.search(r'κ_(\d+)', op_str)
    if match:
        return int(match.group(1))
    else:
        raise ValueError(f'cannot find index in {op_str}')
    

In [47]:
def wick_expansion(op):
    '''
    op: list of operators
    Assume the input operator is in the order of c^dagger c c^dagger c
    '''
    hartree=(expval[(Dagger(op[0]),op[1])]*op[2]*op[3] + op[0]*op[1]*expval[(Dagger(op[2]),op[3])])* delta[(_idx(op[0]),_idx(op[1]))]*delta[(_idx(op[2]),_idx(op[3]))]
    hartree_exp=expval[(Dagger(op[0]),op[1])]* expval[(Dagger(op[2]),op[3])]* delta[(_idx(op[0]),_idx(op[1]))]*delta[(_idx(op[2]),_idx(op[3]))]
    fock=(-expval[(Dagger(op[0]),op[3])]*op[2]*op[1] - op[0]*op[3]*expval[(Dagger(op[2]),op[1])] ) *delta[(_idx(op[0]),_idx(op[3]))]*delta[(_idx(op[2]),_idx(op[1]))]
    fock_exp=-expval[(Dagger(op[0]),op[3])]* expval[(Dagger(op[2]),op[1])] *delta[(_idx(op[0]),_idx(op[3]))]*delta[(_idx(op[2]),_idx(op[1]))]
    return hartree, fock,hartree_exp,fock_exp


In [48]:
def extract_op(h):
    '''
    h: expresion
    '''
    args=h.args
    op_list=[op for op in args if type(op) is type(psil_vec[1][0]) or type(op) is type(psil_d_vec[1][0]) or type(op) is type(psil_d_vec[1][0]) ]
    num_list=list(set(args)-set(op_list))
    if len(num_list)==0:
        num_list=[1]
    return num_list,op_list

In [49]:
def perform_wicks(h,delta=True):
    '''
    h: expression
    '''
    num_list,op_list=extract_op(h)
    hartree, fock,hartree_exp,fock_exp=wick_expansion(op_list)
    num=h.func(*num_list)
    return num*hartree, num*fock,num*hartree_exp,num*fock_exp

In [50]:
def perform_wicks_all(h):
    hartree_list,fock_list,hartree_exp_list,fock_exp_list=list(zip(*[perform_wicks(s) for s in h.args]))
    return h.func(*hartree_list),h.func(*fock_list),h.func(*hartree_exp_list), h.func(*fock_exp_list)

In [51]:
delta={}
delta={(i,j):Symbol(rf'\delta_{{{i},{j}}}',real=True) for i in range(1,5) for j in range(1,5) if i!=j}

In [52]:
delta[(1,2)]

\delta_{1,2}

In [53]:
H10_pm_proj_H,H10_pm_proj_F,H10_pm_proj_H_exp,H10_pm_proj_F_exp=perform_wicks_all(H10_pm_proj)

In [54]:
def extract_delta(h):
    delta_list=[]
    non_delta_list=[]
    for op in h.args:
        match= re.search(r'\\delta_{(\d+),(\d+)}', str(op))
        if match:
            delta_list.append((int(match.group(1)),int(match.group(2))))
        else:
            non_delta_list.append(op)
    return delta_list,h.func(*non_delta_list)

In [55]:
def sum_delta(h):
    delta_list,non_delta_list=extract_delta(h)
    r_theta=[]
    r_psil=[]
    r_cl=[]
    r_order=[]
    r_kappa=[]

    for i,j in delta_list:
        psi_i_list=[(p1,p2) for p1 in psil_vec[i] for p2 in psil_vec[j]]
        psi_j_list=[(p1,p2) for p1 in psil_vec[i] for p2 in psil_vec[i]]
        c_i_list=[(p1,p2) for p1 in cl_vec[i] for p2 in cl_vec[j]]
        c_j_list=[(p1,p2) for p1 in cl_vec[i] for p2 in cl_vec[i]]
        r_theta.append((theta[j],theta[i]))
        r_psil=r_psil+list(zip(psil_vec[j],psil_vec[i]))
        r_cl=r_cl+list(zip(cl_vec[j],cl_vec[i]))
        r_order=r_order+[(expval[psi_i],expval[psi_j]) for psi_i,psi_j in zip(psi_i_list,psi_j_list)] + [(expval[c_i],expval[c_j]) for c_i,c_j in zip(c_i_list,c_j_list)]
        r_kappa.append((κ[j],κ[i]))
    
    rules=r_theta+r_psil+r_cl+r_order+r_kappa


    return non_delta_list.subs(rules)

In [32]:
κ={i:Symbol(f'κ_{i}') for i in range(1,5)}

In [57]:
def generate_relabel_rule(i,j):
    '''
    substitute i to j
    '''
    psi_i_list=[(p1,p2) for p1 in psil_vec[i] for p2 in psil_vec[i]]
    psi_j_list=[(p1,p2) for p1 in psil_vec[j] for p2 in psil_vec[j]]
    c_i_list=[(p1,p2) for p1 in cl_vec[i] for p2 in cl_vec[i]]
    c_j_list=[(p1,p2) for p1 in cl_vec[j] for p2 in cl_vec[j]]

    return [(theta[i],theta[j])]+ [(κ[i],κ[j])]+list(zip(psil_vec[i],psil_vec[j])) + list(zip(psil_d_vec[i],psil_d_vec[j])) + [(expval[psi_i],expval[psi_j]) for psi_i,psi_j in zip(psi_i_list,psi_j_list)] + list(zip(cl_vec[i],cl_vec[j])) + list(zip(cl_d_vec[i],cl_d_vec[j])) + [(expval[c_i],expval[c_j]) for c_i,c_j in zip(c_i_list,c_j_list)]
    

In [58]:
rule32=generate_relabel_rule(3,2)
rule13=generate_relabel_rule(1,3)
rule21=generate_relabel_rule(2,1)
def relabel_idx(h):
    '''
    Change psi_dagger (c_dagger) /psi (c)-> κ1
    and order parameter -> κ3
    use k2 to transit
    κ3 -> κ2
    κ1 -> κ3
    κ2 -> κ1
    '''
    
    if set(psil_vec[3]).intersection(set(h.args)) or set(cl_vec[3]).intersection(set(h.args)):
        return h.subs(rule32).subs(rule13).subs(rule21)
    else:
        return h


In [59]:
def sum_delta_all(h,relabel=False):
    if relabel:
        return h.func(*[relabel_idx(sum_delta(h1)) for h1 in h.args])
    else:
        return h.func(*[sum_delta(h1) for h1 in h.args])

In [60]:
H10_pm_proj_F_bilinear=sum_delta_all(H10_pm_proj_F.expand(),relabel=True)

In [61]:
rules_intravalley=[(expval[(psi["".join([λ1,l1,σ1,'3'])],psi["".join([λ2,l2,σ2,'3'])])],0) for λ1,λ2 in ["-+","+-"] for l1 in "-+" for l2 in "-+" for σ1 in 'ud' for σ2 in 'ud']+[(expval[(c["".join([λ1,l1,σ1,'3'])],c["".join([λ2,l2,σ2,'3'])])],0) for λ1,λ2 in ["-+","+-"] for l1 in "-+" for l2 in "-+" for σ1 in 'ud' for σ2 in 'ud']

In [62]:
rules_intervalley=[(expval[(psi["".join([λ,l1,σ1,'3'])],psi["".join([λ,l2,σ2,'3'])])],0) for λ in "-+" for l1 in "-+" for l2 in "-+" for σ1 in 'ud' for σ2 in 'ud']+[(expval[(c["".join([λ,l1,σ1,'3'])],c["".join([λ,l2,σ2,'3'])])],0) for λ in "-+" for l1 in "-+" for l2 in "-+" for σ1 in 'ud' for σ2 in 'ud']

In [63]:
H10_intra=H10_pm_proj_F_bilinear.subs(rules_intravalley).powsimp().rewrite(cos).expand()

In [64]:
H10_intra

-\langle c_{+K,+,↓,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle*Dagger(c_{+K,-,↑,κ_1})*c_{+K,+,↓,κ_1}/2 + \langle c_{+K,+,↓,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle*Dagger(c_{-K,-,↑,κ_1})*c_{-K,+,↓,κ_1}/2 - \langle c_{+K,-,↑,κ_3}^\dagger c_{+K,+,↓,κ_3} \rangle*Dagger(c_{+K,+,↓,κ_1})*c_{+K,-,↑,κ_1}/2 + \langle c_{+K,-,↑,κ_3}^\dagger c_{+K,+,↓,κ_3} \rangle*Dagger(c_{-K,+,↓,κ_1})*c_{-K,-,↑,κ_1}/2 + \langle c_{-K,+,↓,κ_3}^\dagger c_{-K,-,↑,κ_3} \rangle*Dagger(c_{+K,-,↑,κ_1})*c_{+K,+,↓,κ_1}/2 - \langle c_{-K,+,↓,κ_3}^\dagger c_{-K,-,↑,κ_3} \rangle*Dagger(c_{-K,-,↑,κ_1})*c_{-K,+,↓,κ_1}/2 + \langle c_{-K,-,↑,κ_3}^\dagger c_{-K,+,↓,κ_3} \rangle*Dagger(c_{+K,+,↓,κ_1})*c_{+K,-,↑,κ_1}/2 - \langle c_{-K,-,↑,κ_3}^\dagger c_{-K,+,↓,κ_3} \rangle*Dagger(c_{-K,+,↓,κ_1})*c_{-K,-,↑,κ_1}/2

In [65]:
H10_inter=H10_pm_proj_F_bilinear.subs(rules_intervalley).powsimp().rewrite(cos).expand()

In [66]:
H10_inter

-\langle c_{+K,+,↓,κ_3}^\dagger c_{-K,-,↑,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})*Dagger(c_{-K,-,↑,κ_1})*c_{+K,+,↓,κ_1}/2 - \langle c_{+K,-,↑,κ_3}^\dagger c_{-K,+,↓,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})*Dagger(c_{-K,+,↓,κ_1})*c_{+K,-,↑,κ_1}/2 - \langle c_{-K,+,↓,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})*Dagger(c_{+K,-,↑,κ_1})*c_{-K,+,↓,κ_1}/2 - \langle c_{-K,-,↑,κ_3}^\dagger c_{+K,+,↓,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})*Dagger(c_{+K,+,↓,κ_1})*c_{-K,-,↑,κ_1}/2

In [67]:
H10_pm_proj_F_exp_const=-sum_delta_all(H10_pm_proj_F_exp)

In [68]:
H10_pm_proj_F_exp_const.subs(rules_intervalley).powsimp().rewrite(cos).expand()

\langle c_{+K,+,↓,κ_3}^\dagger c_{-K,-,↑,κ_3} \rangle*\langle c_{-K,-,↑,κ_1}^\dagger c_{+K,+,↓,κ_1} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})/2 + \langle c_{+K,-,↑,κ_1}^\dagger c_{-K,+,↓,κ_1} \rangle*\langle c_{-K,+,↓,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})/2

In [69]:
H10_pm_proj_F_exp_const.subs(rules_intravalley).powsimp().rewrite(cos).expand()

\langle c_{+K,+,↓,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle*\langle c_{+K,-,↑,κ_1}^\dagger c_{+K,+,↓,κ_1} \rangle/2 - \langle c_{+K,+,↓,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle*\langle c_{-K,-,↑,κ_1}^\dagger c_{-K,+,↓,κ_1} \rangle/2 - \langle c_{+K,-,↑,κ_1}^\dagger c_{+K,+,↓,κ_1} \rangle*\langle c_{-K,+,↓,κ_3}^\dagger c_{-K,-,↑,κ_3} \rangle/2 + \langle c_{-K,+,↓,κ_3}^\dagger c_{-K,-,↑,κ_3} \rangle*\langle c_{-K,-,↑,κ_1}^\dagger c_{-K,+,↓,κ_1} \rangle/2

In [298]:
K['+']

Matrix([
[sqrt(3)/3],
[        0]])

In [159]:
z4=zeros(4,4)
mat4={}
for i in range(4):
    for j in range(4):
        tmp=z4.copy()
        tmp[i,j]=1
        mat4[(i,j)]=tmp.copy()

In [173]:
ε={"".join([λ,l,σ,str(k)]): Operator(f'ε_{{{λ}K,{l},{spin(σ)},κ_{k}}}') for λ in '-+' for l in '-+' for σ in 'ud' for k in range(4)}

In [174]:
ek=sum([ε[key]*Dagger(c[key])*c[key] for key in ['+-u1','--u1','++d1','-+d1']])
# diag(ε['+u1'],ε['-u1'],ε['+d1'],ε['-d1'])

In [299]:
ek

ε_{+K,+,↓,κ_1}*Dagger(c_{+K,+,↓,κ_1})*c_{+K,+,↓,κ_1} + ε_{+K,-,↑,κ_1}*Dagger(c_{+K,-,↑,κ_1})*c_{+K,-,↑,κ_1} + ε_{-K,+,↓,κ_1}*Dagger(c_{-K,+,↓,κ_1})*c_{-K,+,↓,κ_1} + ε_{-K,-,↑,κ_1}*Dagger(c_{-K,-,↑,κ_1})*c_{-K,-,↑,κ_1}

In [198]:
cl_eff_d_vec=lambda order_basis: [cl_d_vec[1][i] for i in order_basis]
cl_eff_vec=lambda order_basis: [cl_vec[1][i] for i in order_basis]

rules_cl2mat_inter=list(zip(Matrix([cl_eff_d_vec([1,6,3,4])[i]* cl_eff_vec([1,6,3,4])[j] for i in range(4) for j in range(4)]),[mat4[(i,j)] for i in range(4) for j in range(4)]))

In [199]:
(H10_inter+ek).subs(rules_cl2mat_inter)

Matrix([
[                                                                           ε_{+K,-,↑,κ_1}, -\langle c_{-K,+,↓,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})/2,                                                                                         0,                                                                                         0],
[-\langle c_{+K,-,↑,κ_3}^\dagger c_{-K,+,↓,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})/2,                                                                            ε_{-K,+,↓,κ_1},                                                                                         0,                                                                                         0],
[                                                                                        0,                                                                                         0,                                                                            ε

In [185]:
cl_eff_vec([1,6,4,3])

[c_{+K,-,↑,κ_1}, c_{-K,+,↓,κ_1}, c_{+K,+,↓,κ_1}, c_{-K,-,↑,κ_1}]

In [194]:
rules_cl2mat_intra=list(zip(Matrix([cl_eff_d_vec([1,4,3,6])[i]* cl_eff_vec([1,4,3,6])[j] for i in range(4) for j in range(4)]),[mat4[(i,j)] for i in range(4) for j in range(4)]))

In [195]:
(H10_intra+ek).subs(rules_cl2mat_intra)

Matrix([
[                                                                                                    ε_{+K,-,↑,κ_1}, -\langle c_{+K,+,↓,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle/2 + \langle c_{-K,+,↓,κ_3}^\dagger c_{-K,-,↑,κ_3} \rangle/2,                                                                                                                 0,                                                                                                                 0],
[-\langle c_{+K,-,↑,κ_3}^\dagger c_{+K,+,↓,κ_3} \rangle/2 + \langle c_{-K,-,↑,κ_3}^\dagger c_{-K,+,↓,κ_3} \rangle/2,                                                                                                     ε_{+K,+,↓,κ_1},                                                                                                                 0,                                                                                                                 0],
[                                                              

In [ ]:
[valley_pairing[idx] for idx in [1,-2]]

# $H_{11}$

In [106]:
nn=Symbol('n')

In [107]:
def convert_valley(s):
    return 1 if s=='+' else -1

In [35]:
# H11=sum([Dagger(c["".join([k1,'A',σ1,'1'])]) * c["".join([k2,'A',σ1,'2'])] * Dagger(c["".join([k3,'B',σ2,'3'])]) * c["".join([k4,'B',σ2,'4'])] * exp(-I*(κ[3]-κ[4])*nn) for k1,k2,k3,k4 in valley_pairing for σ1 in 'ud' for σ2 in 'ud'])

In [129]:
# H11=sum([Dagger(c["".join([k1,'A',σ1,'1'])]) * c["".join([k2,'A',σ1,'2'])] * Dagger(c["".join([k3,'B',σ2,'3'])]) * c["".join([k4,'B',σ2,'4'])] * exp(-I* ((convert_valley(k3)-convert_valley(k4))*4*pi/sqrt(3)*K['+']).dot(n[i])) for k1,k2,k3,k4 in valley_pairing for σ1 in 'ud' for σ2 in 'ud' for i in range(3)]).subs(rules_c2psi).rewrite(cos).expand()
H11=sum([Dagger(c["".join([k1,'A',σ1,'1'])]) * c["".join([k2,'A',σ1,'2'])] * Dagger(c["".join([k3,'B',σ2,'3'])]) * c["".join([k4,'B',σ2,'4'])] * exp(-I* ((convert_valley(k3)-convert_valley(k4))*4*pi/sqrt(3)*K['+']).dot(n[i])) for k1,k2,k3,k4 in valley_pairing for σ1 in 'ud' for σ2 in 'ud' for i in range(3)]).rewrite(cos).expand()

In [130]:
# H11_pm=H11.subs(rules_c2psi).subs(rules_psi2psil).expand()
H11_pm=H11.subs(rules_c2cl).expand()

In [131]:
# H11_pm_proj=H11_pm.subs(rules_proj)
H11_pm_proj=H11_pm.subs(rules_proj_3)

In [ ]:
H11_pm_proj

3*Dagger(c_{+K,-,↑,κ_1})*c_{+K,-,↑,κ_2}*Dagger(c_{-K,-,↑,κ_3})*c_{-K,-,↑,κ_4}/4 + 3*exp(-I*\theta_{κ_3})*exp(I*\theta_{κ_4})*Dagger(c_{+K,-,↑,κ_1})*c_{+K,-,↑,κ_2}*Dagger(c_{+K,-,↑,κ_3})*c_{+K,-,↑,κ_4}/4 + 3*exp(-I*\theta_{κ_1})*exp(I*\theta_{κ_2})*Dagger(c_{-K,-,↑,κ_1})*c_{-K,-,↑,κ_2}*Dagger(c_{-K,-,↑,κ_3})*c_{-K,-,↑,κ_4}/4 + 3*exp(-I*\theta_{κ_1})*exp(I*\theta_{κ_2})*exp(-I*\theta_{κ_3})*exp(I*\theta_{κ_4})*Dagger(c_{-K,-,↑,κ_1})*c_{-K,-,↑,κ_2}*Dagger(c_{+K,-,↑,κ_3})*c_{+K,-,↑,κ_4}/4

In [114]:
H11_pm_proj

3*exp(I*\theta_{κ_1}/2)*exp(-I*\theta_{κ_2}/2)*exp(-I*\theta_{κ_3}/2)*exp(I*\theta_{κ_4}/2)*Dagger(c_{+K,-,↑,κ_1})*c_{+K,-,↑,κ_2}*Dagger(c_{+K,-,↑,κ_3})*c_{+K,-,↑,κ_4}/4 + 3*exp(I*\theta_{κ_1}/2)*exp(-I*\theta_{κ_2}/2)*exp(-I*\theta_{κ_3}/2)*exp(I*\theta_{κ_4}/2)*Dagger(c_{+K,-,↑,κ_1})*c_{+K,-,↑,κ_2}*Dagger(c_{-K,-,↑,κ_3})*c_{-K,-,↑,κ_4}/4 + 3*exp(I*\theta_{κ_1}/2)*exp(-I*\theta_{κ_2}/2)*exp(-I*\theta_{κ_3}/2)*exp(I*\theta_{κ_4}/2)*Dagger(c_{-K,-,↑,κ_1})*c_{-K,-,↑,κ_2}*Dagger(c_{+K,-,↑,κ_3})*c_{+K,-,↑,κ_4}/4 + 3*exp(I*\theta_{κ_1}/2)*exp(-I*\theta_{κ_2}/2)*exp(-I*\theta_{κ_3}/2)*exp(I*\theta_{κ_4}/2)*Dagger(c_{-K,-,↑,κ_1})*c_{-K,-,↑,κ_2}*Dagger(c_{-K,-,↑,κ_3})*c_{-K,-,↑,κ_4}/4

In [75]:
extract_op(H11_pm_proj.args[0])

([3/4],
 [Dagger(c_{+K,+,↓,κ_1}),
  c_{+K,+,↓,κ_2},
  Dagger(c_{-K,+,↓,κ_3}),
  c_{-K,+,↓,κ_4}])

In [76]:
H11_pm_proj_H,H11_pm_proj_F,H11_pm_proj_H_exp,H11_pm_proj_F_exp=perform_wicks_all(H11_pm_proj)

In [77]:
rules_spinmix=[(expval[(psi["".join([λ1,l1,σ,k1])],psi["".join([λ2,l2,σ,k2])])],0) for λ1 in '-+' for λ2 in '-+' for l1 in "-+" for l2 in "-+" for σ in 'ud' for k1 in '1234' for k2 in '1234']+[(expval[(c["".join([λ1,l1,σ,k1])],c["".join([λ2,l2,σ,k2])])],0) for λ1 in '-+' for λ2 in '-+' for l1 in "-+" for l2 in "-+" for σ in 'ud' for k1 in '1234' for k2 in '1234']

In [78]:
H11_pm_proj_F_spinmix=H11_pm_proj_F.expand()
# .subs(rules_spinmix)

In [79]:
H11_pm_proj_F_bilinear=sum_delta_all(H11_pm_proj_F_spinmix,relabel=True)

In [80]:
H11_intra=H11_pm_proj_F_bilinear.subs(rules_intravalley).powsimp().rewrite(cos).expand()

In [81]:
H11_inter=H11_pm_proj_F_bilinear.subs(rules_intervalley).powsimp().rewrite(exp).expand()

In [82]:
H11_pm_proj_F_exp_const=-sum_delta_all(H11_pm_proj_F_exp)

In [89]:
H11_pm_proj_F_exp_const.subs(rules_spinmix).subs(rules_intervalley).powsimp().rewrite(cos).expand()

3*\langle c_{+K,+,↓,κ_1}^\dagger c_{-K,-,↑,κ_1} \rangle*\langle c_{-K,-,↑,κ_3}^\dagger c_{+K,+,↓,κ_3} \rangle/4 + 3*\langle c_{+K,+,↓,κ_3}^\dagger c_{-K,-,↑,κ_3} \rangle*\langle c_{-K,-,↑,κ_1}^\dagger c_{+K,+,↓,κ_1} \rangle/4 + 3*\langle c_{+K,-,↑,κ_1}^\dagger c_{-K,+,↓,κ_1} \rangle*\langle c_{-K,+,↓,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle/4 + 3*\langle c_{+K,-,↑,κ_3}^\dagger c_{-K,+,↓,κ_3} \rangle*\langle c_{-K,+,↓,κ_1}^\dagger c_{+K,-,↑,κ_1} \rangle/4

In [90]:
H11_pm_proj_F_exp_const.subs(rules_spinmix).subs(rules_intravalley)

3*\langle c_{+K,+,↓,κ_1}^\dagger c_{+K,-,↑,κ_1} \rangle*\langle c_{+K,-,↑,κ_3}^\dagger c_{+K,+,↓,κ_3} \rangle*exp(I*\theta_{κ_1})*exp(-I*\theta_{κ_3})/4 + 3*\langle c_{+K,+,↓,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle*\langle c_{+K,-,↑,κ_1}^\dagger c_{+K,+,↓,κ_1} \rangle*exp(I*\theta_{κ_1})*exp(-I*\theta_{κ_3})/4 + 3*\langle c_{-K,+,↓,κ_1}^\dagger c_{-K,-,↑,κ_1} \rangle*\langle c_{-K,-,↑,κ_3}^\dagger c_{-K,+,↓,κ_3} \rangle*exp(-I*\theta_{κ_1})*exp(I*\theta_{κ_3})/4 + 3*\langle c_{-K,+,↓,κ_3}^\dagger c_{-K,-,↑,κ_3} \rangle*\langle c_{-K,-,↑,κ_1}^\dagger c_{-K,+,↓,κ_1} \rangle*exp(-I*\theta_{κ_1})*exp(I*\theta_{κ_3})/4

In [85]:
H11_intra

-3*\langle c_{+K,+,↓,κ_3}^\dagger c_{+K,+,↓,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})*Dagger(c_{+K,+,↓,κ_1})*c_{+K,+,↓,κ_1}/2 - 3*\langle c_{+K,+,↓,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})*Dagger(c_{+K,-,↑,κ_1})*c_{+K,+,↓,κ_1}/2 - 3*\langle c_{+K,-,↑,κ_3}^\dagger c_{+K,+,↓,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})*Dagger(c_{+K,+,↓,κ_1})*c_{+K,-,↑,κ_1}/2 - 3*\langle c_{+K,-,↑,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})*Dagger(c_{+K,-,↑,κ_1})*c_{+K,-,↑,κ_1}/2 - 3*\langle c_{-K,+,↓,κ_3}^\dagger c_{-K,+,↓,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})*Dagger(c_{-K,+,↓,κ_1})*c_{-K,+,↓,κ_1}/2 - 3*\langle c_{-K,+,↓,κ_3}^\dagger c_{-K,-,↑,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})*Dagger(c_{-K,-,↑,κ_1})*c_{-K,+,↓,κ_1}/2 - 3*\langle c_{-K,-,↑,κ_3}^\dagger c_{-K,+,↓,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})*Dagger(c_{-K,+,↓,κ_1})*c_{-K,-,↑,κ_1}/2 - 3*\langle c_{-K,-,↑,κ_3}^\dagger c_{-K,-,↑,κ_3} \rangle*cos(\theta_{κ_1} - \theta_

In [86]:
H11_inter

-3*\langle c_{+K,+,↓,κ_3}^\dagger c_{-K,+,↓,κ_3} \rangle*Dagger(c_{-K,+,↓,κ_1})*c_{+K,+,↓,κ_1}/2 - 3*\langle c_{+K,+,↓,κ_3}^\dagger c_{-K,-,↑,κ_3} \rangle*Dagger(c_{-K,-,↑,κ_1})*c_{+K,+,↓,κ_1}/2 - 3*\langle c_{+K,-,↑,κ_3}^\dagger c_{-K,+,↓,κ_3} \rangle*Dagger(c_{-K,+,↓,κ_1})*c_{+K,-,↑,κ_1}/2 - 3*\langle c_{+K,-,↑,κ_3}^\dagger c_{-K,-,↑,κ_3} \rangle*Dagger(c_{-K,-,↑,κ_1})*c_{+K,-,↑,κ_1}/2 - 3*\langle c_{-K,+,↓,κ_3}^\dagger c_{+K,+,↓,κ_3} \rangle*Dagger(c_{+K,+,↓,κ_1})*c_{-K,+,↓,κ_1}/2 - 3*\langle c_{-K,+,↓,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle*Dagger(c_{+K,-,↑,κ_1})*c_{-K,+,↓,κ_1}/2 - 3*\langle c_{-K,-,↑,κ_3}^\dagger c_{+K,+,↓,κ_3} \rangle*Dagger(c_{+K,+,↓,κ_1})*c_{-K,-,↑,κ_1}/2 - 3*\langle c_{-K,-,↑,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle*Dagger(c_{+K,-,↑,κ_1})*c_{-K,-,↑,κ_1}/2

In [200]:
(H11_intra+ek).subs(rules_cl2mat_intra)

Matrix([
[-3*\langle c_{+K,-,↑,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})/2 + ε_{+K,-,↑,κ_1},                  -3*\langle c_{+K,+,↓,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})/2,                                                                                                            0,                                                                                                            0],
[                 -3*\langle c_{+K,-,↑,κ_3}^\dagger c_{+K,+,↓,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})/2, -3*\langle c_{+K,+,↓,κ_3}^\dagger c_{+K,+,↓,κ_3} \rangle*cos(\theta_{κ_1} - \theta_{κ_3})/2 + ε_{+K,+,↓,κ_1},                                                                                                            0,                                                                                                            0],
[                                                                                                          

In [201]:
(H11_inter+ek).subs(rules_cl2mat_inter)

Matrix([
[                                            ε_{+K,-,↑,κ_1}, -3*\langle c_{-K,+,↓,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle/2, -3*\langle c_{-K,-,↑,κ_3}^\dagger c_{+K,-,↑,κ_3} \rangle/2,                                                          0],
[-3*\langle c_{+K,-,↑,κ_3}^\dagger c_{-K,+,↓,κ_3} \rangle/2,                                             ε_{-K,+,↓,κ_1},                                                          0, -3*\langle c_{+K,+,↓,κ_3}^\dagger c_{-K,+,↓,κ_3} \rangle/2],
[-3*\langle c_{+K,-,↑,κ_3}^\dagger c_{-K,-,↑,κ_3} \rangle/2,                                                          0,                                             ε_{-K,-,↑,κ_1}, -3*\langle c_{+K,+,↓,κ_3}^\dagger c_{-K,-,↑,κ_3} \rangle/2],
[                                                         0, -3*\langle c_{-K,+,↓,κ_3}^\dagger c_{+K,+,↓,κ_3} \rangle/2, -3*\langle c_{-K,-,↑,κ_3}^\dagger c_{+K,+,↓,κ_3} \rangle/2,                                             ε_{+K,+,↓,κ_1}]])

In [ ]:
K['+'].dot(n[1])*4*pi/sqrt(3)

In [ ]:
(Matrix([[cos(rad(120)),-sin(rad(120))],[sin(rad(120)),cos(rad(120))]])*K['-']).dot(n[1])*4*pi/sqrt(3)

In [ ]:
(Matrix([[cos(rad(120)),sin(rad(120))],[-sin(rad(120)),cos(rad(120))]])*K['+']).dot(n[1])*4*pi/sqrt(3)

In [ ]:
sum([Dagger(c["".join([k1,'A',σ1])]) * c["".join([k2,'A',σ1])] * Dagger(c["".join([k3,'B',σ2])]) * c["".join([k4,'B',σ2])]  for k1,k2,k3,k4 in [valley_pairing[idx] for idx in [2,-3]] for σ1 in 'u' for σ2 in 'd'])

In [ ]:
sum([Dagger(c["".join([k1,'A',σ1])]) * c["".join([k2,'A',σ1])] * Dagger(c["".join([k3,'B',σ2])]) * c["".join([k4,'B',σ2])]  for k1,k2,k3,k4 in [valley_pairing[idx] for idx in [2,-3]] for σ1 in 'u' for σ2 in 'd']).subs(rules_c2psi)
# * exp(-I* ((convert_valley(k3)-convert_valley(k4))*4*pi/sqrt(3)*K['+']).dot(n[i]))
# for l in 'AB' for σ1 in 'ud' for σ2 in 'ud'

# Check order parameter

In [39]:
O={}
O={(i,j): Symbol(f'O_{{{i},{j}}}') for i in range(1,3) for j in range(1,3)}

In [40]:
O_mat=Matrix([[O[i,j] for j in range(1,3)] for i in range(1,3)])

In [41]:
kronecker_product(sigma['-'],O_mat)

Matrix([
[      0,       0, 0, 0],
[      0,       0, 0, 0],
[O_{1,1}, O_{1,2}, 0, 0],
[O_{2,1}, O_{2,2}, 0, 0]])

In [42]:
P=Matrix([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])

In [43]:
OO=P*kronecker_product(sigma['-'],O_mat)*P
# OO=Matrix([[0,0,0,0],[O[1,1],0,0,O[1,2]],[O[2,1],0,0,O[2,2]],[0,0,0,0]])

In [44]:
OO

Matrix([
[      0, 0,       0, 0],
[O_{1,1}, 0, O_{1,2}, 0],
[      0, 0,       0, 0],
[O_{2,1}, 0, O_{2,2}, 0]])

In [45]:
OOO=kronecker_product(sigma['+'],OO)+ kronecker_product(sigma['+'],OO).H

In [46]:
OOO

Matrix([
[0,                  0, 0,                  0,       0, 0,       0, 0],
[0,                  0, 0,                  0, O_{1,1}, 0, O_{1,2}, 0],
[0,                  0, 0,                  0,       0, 0,       0, 0],
[0,                  0, 0,                  0, O_{2,1}, 0, O_{2,2}, 0],
[0, conjugate(O_{1,1}), 0, conjugate(O_{2,1}),       0, 0,       0, 0],
[0,                  0, 0,                  0,       0, 0,       0, 0],
[0, conjugate(O_{1,2}), 0, conjugate(O_{2,2}),       0, 0,       0, 0],
[0,                  0, 0,                  0,       0, 0,       0, 0]])

In [47]:
WW[1].H * OOO* WW[1]

Matrix([
[                                         0,                                         0,                                        0,                                          0,                       O_{1,1}/2, O_{1,1}*exp(I*\theta_{κ_1})/2, -O_{1,2}*exp(I*\theta_{κ_1})/2,                       O_{1,2}/2],
[                                         0,                                         0,                                        0,                                          0, -O_{1,1}*exp(-I*\theta_{κ_1})/2,                    -O_{1,1}/2,                      O_{1,2}/2, -O_{1,2}*exp(-I*\theta_{κ_1})/2],
[                                         0,                                         0,                                        0,                                          0,  O_{2,1}*exp(-I*\theta_{κ_1})/2,                     O_{2,1}/2,                     -O_{2,2}/2,  O_{2,2}*exp(-I*\theta_{κ_1})/2],
[                                         0,                                   

In [48]:
rules_intra=[(O[(1,2)],0),(O[(2,1)],0)]
rules_inter=[(O[(1,1)],0),(O[(2,2)],0)]

In [49]:
# COO=(M.T *WW[1].H * OOO* WW[1] * M)
COO=WW[1].H * OOO* WW[1]

In [50]:
COO[:4,4:]

Matrix([
[                      O_{1,1}/2, O_{1,1}*exp(I*\theta_{κ_1})/2, -O_{1,2}*exp(I*\theta_{κ_1})/2,                       O_{1,2}/2],
[-O_{1,1}*exp(-I*\theta_{κ_1})/2,                    -O_{1,1}/2,                      O_{1,2}/2, -O_{1,2}*exp(-I*\theta_{κ_1})/2],
[ O_{2,1}*exp(-I*\theta_{κ_1})/2,                     O_{2,1}/2,                     -O_{2,2}/2,  O_{2,2}*exp(-I*\theta_{κ_1})/2],
[                      O_{2,1}/2, O_{2,1}*exp(I*\theta_{κ_1})/2, -O_{2,2}*exp(I*\theta_{κ_1})/2,                       O_{2,2}/2]])

In [51]:
COO.subs(rules_intra)

Matrix([
[                                        0,                                         0,                                        0,                                          0,                       O_{1,1}/2, O_{1,1}*exp(I*\theta_{κ_1})/2,                              0,                              0],
[                                        0,                                         0,                                        0,                                          0, -O_{1,1}*exp(-I*\theta_{κ_1})/2,                    -O_{1,1}/2,                              0,                              0],
[                                        0,                                         0,                                        0,                                          0,                               0,                             0,                     -O_{2,2}/2, O_{2,2}*exp(-I*\theta_{κ_1})/2],
[                                        0,                                         0

In [52]:
COO.subs(rules_inter)

Matrix([
[                                         0,                                         0,                                        0,                                         0,                              0,                             0, -O_{1,2}*exp(I*\theta_{κ_1})/2,                       O_{1,2}/2],
[                                         0,                                         0,                                        0,                                         0,                              0,                             0,                      O_{1,2}/2, -O_{1,2}*exp(-I*\theta_{κ_1})/2],
[                                         0,                                         0,                                        0,                                         0, O_{2,1}*exp(-I*\theta_{κ_1})/2,                     O_{2,1}/2,                              0,                               0],
[                                         0,                                         

In [53]:
cr={"".join([l,σ]):Operator(f'c_{{{l},{spin(σ)}}}') for l in 'AB' for σ in 'ud'}

In [54]:
cc=Matrix([[cr["".join([l,σ])] for l in 'AB'] for σ in 'ud'])
cc_d=Matrix([[Dagger(cr["".join([l,σ])]) for σ in 'ud'] for l in 'AB'])

In [55]:
cc_d*sigma['x']*cc

Matrix([
[Dagger(c_{A,↑})*c_{A,↓} + Dagger(c_{A,↓})*c_{A,↑}, Dagger(c_{A,↑})*c_{B,↓} + Dagger(c_{A,↓})*c_{B,↑}],
[Dagger(c_{B,↑})*c_{A,↓} + Dagger(c_{B,↓})*c_{A,↑}, Dagger(c_{B,↑})*c_{B,↓} + Dagger(c_{B,↓})*c_{B,↑}]])

In [219]:
cc_d*sigma['y']*cc

Matrix([
[-I*Dagger(c_{A,↑})*c_{A,↓} + I*Dagger(c_{A,↓})*c_{A,↑}, -I*Dagger(c_{A,↑})*c_{B,↓} + I*Dagger(c_{A,↓})*c_{B,↑}],
[-I*Dagger(c_{B,↑})*c_{A,↓} + I*Dagger(c_{B,↓})*c_{A,↑}, -I*Dagger(c_{B,↑})*c_{B,↓} + I*Dagger(c_{B,↓})*c_{B,↑}]])

In [56]:
P

Matrix([
[1, 0, 0, 0],
[0, 0, 1, 0],
[0, 1, 0, 0],
[0, 0, 0, 1]])

In [57]:
P*c_vec[1].reshape(4,2)

Matrix([
[c_{+K,A,↑,κ_1}, c_{+K,B,↑,κ_1}],
[c_{+K,A,↓,κ_1}, c_{+K,B,↓,κ_1}],
[c_{-K,A,↑,κ_1}, c_{-K,B,↑,κ_1}],
[c_{-K,A,↓,κ_1}, c_{-K,B,↓,κ_1}]])

In [58]:
(P*c_d_vec[1].reshape(4,2)).T

Matrix([
[Dagger(c_{+K,A,↑,κ_1}), Dagger(c_{+K,A,↓,κ_1}), Dagger(c_{-K,A,↑,κ_1}), Dagger(c_{-K,A,↓,κ_1})],
[Dagger(c_{+K,B,↑,κ_1}), Dagger(c_{+K,B,↓,κ_1}), Dagger(c_{-K,B,↑,κ_1}), Dagger(c_{-K,B,↓,κ_1})]])

In [59]:
Kr=symbols('Kr')

In [60]:
s={spin:Matrix([[sigma[spin],sigma[spin]*exp(-2*I*Kr)],[sigma[spin]*exp(2*I*Kr),sigma[spin]]]) for spin in 'xy'}

In [61]:
diag(*[exp(I*K[valley].dot(n[2])*4*pi/sqrt(3)) for valley in '++--'])

Matrix([
[exp(2*I*pi/3),             0,              0,              0],
[            0, exp(2*I*pi/3),              0,              0],
[            0,             0, exp(-2*I*pi/3),              0],
[            0,             0,              0, exp(-2*I*pi/3)]])

In [62]:
S={(spin,n_idx):((P*c_d_vec[1].reshape(4,2)).T * s[spin] * diag(*[exp(I*K[valley].dot(n[n_idx])*4*pi/sqrt(3)) for valley in '++--']) * (P*c_vec[1].reshape(4,2))).expand() for spin in 'xy' for n_idx in range(3)}

In [63]:
S[('x',0)]

Matrix([
[exp(2*I*Kr)*Dagger(c_{-K,A,↑,κ_1})*c_{+K,A,↓,κ_1} + exp(2*I*Kr)*Dagger(c_{-K,A,↓,κ_1})*c_{+K,A,↑,κ_1} + Dagger(c_{+K,A,↑,κ_1})*c_{+K,A,↓,κ_1} + Dagger(c_{+K,A,↓,κ_1})*c_{+K,A,↑,κ_1} + Dagger(c_{-K,A,↑,κ_1})*c_{-K,A,↓,κ_1} + Dagger(c_{-K,A,↓,κ_1})*c_{-K,A,↑,κ_1} + exp(-2*I*Kr)*Dagger(c_{+K,A,↑,κ_1})*c_{-K,A,↓,κ_1} + exp(-2*I*Kr)*Dagger(c_{+K,A,↓,κ_1})*c_{-K,A,↑,κ_1}, exp(2*I*Kr)*Dagger(c_{-K,A,↑,κ_1})*c_{+K,B,↓,κ_1} + exp(2*I*Kr)*Dagger(c_{-K,A,↓,κ_1})*c_{+K,B,↑,κ_1} + Dagger(c_{+K,A,↑,κ_1})*c_{+K,B,↓,κ_1} + Dagger(c_{+K,A,↓,κ_1})*c_{+K,B,↑,κ_1} + Dagger(c_{-K,A,↑,κ_1})*c_{-K,B,↓,κ_1} + Dagger(c_{-K,A,↓,κ_1})*c_{-K,B,↑,κ_1} + exp(-2*I*Kr)*Dagger(c_{+K,A,↑,κ_1})*c_{-K,B,↓,κ_1} + exp(-2*I*Kr)*Dagger(c_{+K,A,↓,κ_1})*c_{-K,B,↑,κ_1}],
[exp(2*I*Kr)*Dagger(c_{-K,B,↑,κ_1})*c_{+K,A,↓,κ_1} + exp(2*I*Kr)*Dagger(c_{-K,B,↓,κ_1})*c_{+K,A,↑,κ_1} + Dagger(c_{+K,B,↑,κ_1})*c_{+K,A,↓,κ_1} + Dagger(c_{+K,B,↓,κ_1})*c_{+K,A,↑,κ_1} + Dagger(c_{-K,B,↑,κ_1})*c_{-K,A,↓,κ_1} + Dagger(c_{-K,B,↓,κ_1})*c_{

In [76]:
cm={λ:Matrix([[c["".join([λ,'Au1'])],c["".join([λ,'Bu1'])]],[c["".join([λ,'Ad1'])],c["".join([λ,'Bd1'])]]]) for λ in '+-'}
cm_d={λ:Matrix([[Dagger(c["".join([λ,'Au1'])]),Dagger(c["".join([λ,'Bu1'])])],[Dagger(c["".join([λ,'Ad1'])]),Dagger(c["".join([λ,'Bd1'])])]]).T for λ in '+-'}

In [108]:
zz=sum([cm_d[λ1]*sigma['x']*cm[λ2]*exp(-I*(convert_valley(λ1)-convert_valley(λ2))*Kr)*exp(-I*convert_valley(λ2)*2*pi/3) for λ1 in '+-' for λ2 in '+-'],zeros(2,2)).expand() 

In [109]:
S[('x',1)]

Matrix([
[exp(-2*I*pi/3)*exp(2*I*Kr)*Dagger(c_{-K,A,↑,κ_1})*c_{+K,A,↓,κ_1} + exp(-2*I*pi/3)*exp(2*I*Kr)*Dagger(c_{-K,A,↓,κ_1})*c_{+K,A,↑,κ_1} + exp(-2*I*pi/3)*Dagger(c_{+K,A,↑,κ_1})*c_{+K,A,↓,κ_1} + exp(-2*I*pi/3)*Dagger(c_{+K,A,↓,κ_1})*c_{+K,A,↑,κ_1} + exp(2*I*pi/3)*Dagger(c_{-K,A,↑,κ_1})*c_{-K,A,↓,κ_1} + exp(2*I*pi/3)*Dagger(c_{-K,A,↓,κ_1})*c_{-K,A,↑,κ_1} + exp(2*I*pi/3)*exp(-2*I*Kr)*Dagger(c_{+K,A,↑,κ_1})*c_{-K,A,↓,κ_1} + exp(2*I*pi/3)*exp(-2*I*Kr)*Dagger(c_{+K,A,↓,κ_1})*c_{-K,A,↑,κ_1}, exp(-2*I*pi/3)*exp(2*I*Kr)*Dagger(c_{-K,A,↑,κ_1})*c_{+K,B,↓,κ_1} + exp(-2*I*pi/3)*exp(2*I*Kr)*Dagger(c_{-K,A,↓,κ_1})*c_{+K,B,↑,κ_1} + exp(-2*I*pi/3)*Dagger(c_{+K,A,↑,κ_1})*c_{+K,B,↓,κ_1} + exp(-2*I*pi/3)*Dagger(c_{+K,A,↓,κ_1})*c_{+K,B,↑,κ_1} + exp(2*I*pi/3)*Dagger(c_{-K,A,↑,κ_1})*c_{-K,B,↓,κ_1} + exp(2*I*pi/3)*Dagger(c_{-K,A,↓,κ_1})*c_{-K,B,↑,κ_1} + exp(2*I*pi/3)*exp(-2*I*Kr)*Dagger(c_{+K,A,↑,κ_1})*c_{-K,B,↓,κ_1} + exp(2*I*pi/3)*exp(-2*I*Kr)*Dagger(c_{+K,A,↓,κ_1})*c_{-K,B,↑,κ_1}],
[exp(-2*I*pi/3)*exp

In [100]:
zz[0,0]

exp(2*I*Kr)*Dagger(c_{-K,A,↑,κ_1})*c_{+K,A,↓,κ_1} + exp(2*I*Kr)*Dagger(c_{-K,A,↓,κ_1})*c_{+K,A,↑,κ_1} + Dagger(c_{+K,A,↑,κ_1})*c_{+K,A,↓,κ_1} + Dagger(c_{+K,A,↓,κ_1})*c_{+K,A,↑,κ_1} + Dagger(c_{-K,A,↑,κ_1})*c_{-K,A,↓,κ_1} + Dagger(c_{-K,A,↓,κ_1})*c_{-K,A,↑,κ_1} + exp(-2*I*Kr)*Dagger(c_{+K,A,↑,κ_1})*c_{-K,A,↓,κ_1} + exp(-2*I*Kr)*Dagger(c_{+K,A,↓,κ_1})*c_{-K,A,↑,κ_1}

In [113]:
S[('x',1)][1,0]-zz[1,0]

0

In [322]:
S[('y',1)]

Matrix([
[-I*exp(-2*I*pi/3)*exp(2*I*Kr)*Dagger(c_{-K,A,↑,κ_1})*c_{+K,A,↓,κ_1} + I*exp(-2*I*pi/3)*exp(2*I*Kr)*Dagger(c_{-K,A,↓,κ_1})*c_{+K,A,↑,κ_1} - I*exp(-2*I*pi/3)*Dagger(c_{+K,A,↑,κ_1})*c_{+K,A,↓,κ_1} + I*exp(-2*I*pi/3)*Dagger(c_{+K,A,↓,κ_1})*c_{+K,A,↑,κ_1} - I*exp(2*I*pi/3)*Dagger(c_{-K,A,↑,κ_1})*c_{-K,A,↓,κ_1} + I*exp(2*I*pi/3)*Dagger(c_{-K,A,↓,κ_1})*c_{-K,A,↑,κ_1} - I*exp(2*I*pi/3)*exp(-2*I*Kr)*Dagger(c_{+K,A,↑,κ_1})*c_{-K,A,↓,κ_1} + I*exp(2*I*pi/3)*exp(-2*I*Kr)*Dagger(c_{+K,A,↓,κ_1})*c_{-K,A,↑,κ_1}, -I*exp(-2*I*pi/3)*exp(2*I*Kr)*Dagger(c_{-K,A,↑,κ_1})*c_{+K,B,↓,κ_1} + I*exp(-2*I*pi/3)*exp(2*I*Kr)*Dagger(c_{-K,A,↓,κ_1})*c_{+K,B,↑,κ_1} - I*exp(-2*I*pi/3)*Dagger(c_{+K,A,↑,κ_1})*c_{+K,B,↓,κ_1} + I*exp(-2*I*pi/3)*Dagger(c_{+K,A,↓,κ_1})*c_{+K,B,↑,κ_1} - I*exp(2*I*pi/3)*Dagger(c_{-K,A,↑,κ_1})*c_{-K,B,↓,κ_1} + I*exp(2*I*pi/3)*Dagger(c_{-K,A,↓,κ_1})*c_{-K,B,↑,κ_1} - I*exp(2*I*pi/3)*exp(-2*I*Kr)*Dagger(c_{+K,A,↑,κ_1})*c_{-K,B,↓,κ_1} + I*exp(2*I*pi/3)*exp(-2*I*Kr)*Dagger(c_{+K,A,↓,κ_1})*c_

In [119]:
rules_O=list(zip(Matrix([c_d_vec[1][i]* c_vec[1][j] for i in range(8) for j in range(8)]),COO.reshape(64,1)))

In [325]:
S[('x',0)].subs(rules_O).subs(rules_intra)

Matrix([
[                                                                                   O_{1,1}/2 - O_{2,2}/2 + conjugate(O_{1,1})/2 - conjugate(O_{2,2})/2, O_{1,1}*exp(I*\theta_{κ_1})/2 + O_{2,2}*exp(-I*\theta_{κ_1})/2 - exp(I*\theta_{κ_1})*conjugate(O_{1,1})/2 - exp(-I*\theta_{κ_1})*conjugate(O_{2,2})/2],
[-O_{1,1}*exp(-I*\theta_{κ_1})/2 - O_{2,2}*exp(I*\theta_{κ_1})/2 + exp(I*\theta_{κ_1})*conjugate(O_{2,2})/2 + exp(-I*\theta_{κ_1})*conjugate(O_{1,1})/2,                                                                                  -O_{1,1}/2 + O_{2,2}/2 - conjugate(O_{1,1})/2 + conjugate(O_{2,2})/2]])

In [329]:
S[('x',1)].subs(rules_O).subs(rules_intra)

Matrix([
[                                                                                   O_{1,1}*exp(-2*I*pi/3)/2 - (-1)**(2/3)*O_{2,2}/2 + exp(-2*I*pi/3)*conjugate(O_{1,1})/2 - (-1)**(2/3)*conjugate(O_{2,2})/2, O_{1,1}*exp(-2*I*pi/3)*exp(I*\theta_{κ_1})/2 + (-1)**(2/3)*O_{2,2}*exp(-I*\theta_{κ_1})/2 - exp(-2*I*pi/3)*exp(I*\theta_{κ_1})*conjugate(O_{1,1})/2 - (-1)**(2/3)*exp(-I*\theta_{κ_1})*conjugate(O_{2,2})/2],
[-O_{1,1}*exp(-2*I*pi/3)*exp(-I*\theta_{κ_1})/2 - (-1)**(2/3)*O_{2,2}*exp(I*\theta_{κ_1})/2 + (-1)**(2/3)*exp(I*\theta_{κ_1})*conjugate(O_{2,2})/2 + exp(-2*I*pi/3)*exp(-I*\theta_{κ_1})*conjugate(O_{1,1})/2,                                                                                  -O_{1,1}*exp(-2*I*pi/3)/2 + (-1)**(2/3)*O_{2,2}/2 - exp(-2*I*pi/3)*conjugate(O_{1,1})/2 + (-1)**(2/3)*conjugate(O_{2,2})/2]])

In [330]:
S[('x',2)].subs(rules_O).subs(rules_intra)

Matrix([
[                                                                                   (-1)**(2/3)*O_{1,1}/2 - O_{2,2}*exp(-2*I*pi/3)/2 + (-1)**(2/3)*conjugate(O_{1,1})/2 - exp(-2*I*pi/3)*conjugate(O_{2,2})/2, (-1)**(2/3)*O_{1,1}*exp(I*\theta_{κ_1})/2 + O_{2,2}*exp(-2*I*pi/3)*exp(-I*\theta_{κ_1})/2 - (-1)**(2/3)*exp(I*\theta_{κ_1})*conjugate(O_{1,1})/2 - exp(-2*I*pi/3)*exp(-I*\theta_{κ_1})*conjugate(O_{2,2})/2],
[-(-1)**(2/3)*O_{1,1}*exp(-I*\theta_{κ_1})/2 - O_{2,2}*exp(-2*I*pi/3)*exp(I*\theta_{κ_1})/2 + exp(-2*I*pi/3)*exp(I*\theta_{κ_1})*conjugate(O_{2,2})/2 + (-1)**(2/3)*exp(-I*\theta_{κ_1})*conjugate(O_{1,1})/2,                                                                                  -(-1)**(2/3)*O_{1,1}/2 + O_{2,2}*exp(-2*I*pi/3)/2 - (-1)**(2/3)*conjugate(O_{1,1})/2 + exp(-2*I*pi/3)*conjugate(O_{2,2})/2]])

In [326]:
S[('y',0)].subs(rules_O).subs(rules_intra)

Matrix([
[                                                                                 -I*O_{1,1}/2 + I*O_{2,2}/2 + I*conjugate(O_{1,1})/2 - I*conjugate(O_{2,2})/2, -I*O_{1,1}*exp(I*\theta_{κ_1})/2 - I*O_{2,2}*exp(-I*\theta_{κ_1})/2 - I*exp(I*\theta_{κ_1})*conjugate(O_{1,1})/2 - I*exp(-I*\theta_{κ_1})*conjugate(O_{2,2})/2],
[I*O_{1,1}*exp(-I*\theta_{κ_1})/2 + I*O_{2,2}*exp(I*\theta_{κ_1})/2 + I*exp(I*\theta_{κ_1})*conjugate(O_{2,2})/2 + I*exp(-I*\theta_{κ_1})*conjugate(O_{1,1})/2,                                                                                    I*O_{1,1}/2 - I*O_{2,2}/2 - I*conjugate(O_{1,1})/2 + I*conjugate(O_{2,2})/2]])

In [327]:
S[('x',0)].subs(rules_O).subs(rules_inter)

Matrix([
[-O_{1,2}*exp(-2*I*Kr)*exp(I*\theta_{κ_1})/2 + O_{2,1}*exp(2*I*Kr)*exp(-I*\theta_{κ_1})/2 - exp(2*I*Kr)*exp(-I*\theta_{κ_1})*conjugate(O_{1,2})/2 + exp(-2*I*Kr)*exp(I*\theta_{κ_1})*conjugate(O_{2,1})/2,                                                                                    O_{1,2}*exp(-2*I*Kr)/2 + O_{2,1}*exp(2*I*Kr)/2 + exp(2*I*Kr)*conjugate(O_{1,2})/2 + exp(-2*I*Kr)*conjugate(O_{2,1})/2],
[                                                                                   O_{1,2}*exp(-2*I*Kr)/2 + O_{2,1}*exp(2*I*Kr)/2 + exp(2*I*Kr)*conjugate(O_{1,2})/2 + exp(-2*I*Kr)*conjugate(O_{2,1})/2, -O_{1,2}*exp(-2*I*Kr)*exp(-I*\theta_{κ_1})/2 + O_{2,1}*exp(2*I*Kr)*exp(I*\theta_{κ_1})/2 - exp(2*I*Kr)*exp(I*\theta_{κ_1})*conjugate(O_{1,2})/2 + exp(-2*I*Kr)*exp(-I*\theta_{κ_1})*conjugate(O_{2,1})/2]])

In [328]:
S[('y',0)].subs(rules_O).subs(rules_inter)

Matrix([
[I*O_{1,2}*exp(-2*I*Kr)*exp(I*\theta_{κ_1})/2 - I*O_{2,1}*exp(2*I*Kr)*exp(-I*\theta_{κ_1})/2 - I*exp(2*I*Kr)*exp(-I*\theta_{κ_1})*conjugate(O_{1,2})/2 + I*exp(-2*I*Kr)*exp(I*\theta_{κ_1})*conjugate(O_{2,1})/2,                                                                                  -I*O_{1,2}*exp(-2*I*Kr)/2 - I*O_{2,1}*exp(2*I*Kr)/2 + I*exp(2*I*Kr)*conjugate(O_{1,2})/2 + I*exp(-2*I*Kr)*conjugate(O_{2,1})/2],
[                                                                                 -I*O_{1,2}*exp(-2*I*Kr)/2 - I*O_{2,1}*exp(2*I*Kr)/2 + I*exp(2*I*Kr)*conjugate(O_{1,2})/2 + I*exp(-2*I*Kr)*conjugate(O_{2,1})/2, I*O_{1,2}*exp(-2*I*Kr)*exp(-I*\theta_{κ_1})/2 - I*O_{2,1}*exp(2*I*Kr)*exp(I*\theta_{κ_1})/2 - I*exp(2*I*Kr)*exp(I*\theta_{κ_1})*conjugate(O_{1,2})/2 + I*exp(-2*I*Kr)*exp(-I*\theta_{κ_1})*conjugate(O_{2,1})/2]])

In [115]:
Re,Im={},{}
Re={(i,j): Symbol(f'a_{{{i},{j}}}',real=True) for i in range(1,3) for j in range(1,3)}
Im={(i,j): Symbol(f'b_{{{i},{j}}}',real=True) for i in range(1,3) for j in range(1,3)}

In [116]:
rules_Oab=[(O[(i,j)],Re[(i,j)]+I*Im[(i,j)]) for i in range(1,3) for j in range(1,3)]

In [235]:
S['x'].subs(rules_O)

Matrix([
[ O_{1,1}/2 - O_{1,2}*exp(-2*I*Kr)*exp(I*\theta_{κ_1})/2 + O_{2,1}*exp(2*I*Kr)*exp(-I*\theta_{κ_1})/2 - O_{2,2}/2 - exp(2*I*Kr)*exp(-I*\theta_{κ_1})*conjugate(O_{1,2})/2 + conjugate(O_{1,1})/2 - conjugate(O_{2,2})/2 + exp(-2*I*Kr)*exp(I*\theta_{κ_1})*conjugate(O_{2,1})/2,  O_{1,1}*exp(I*\theta_{κ_1})/2 + O_{1,2}*exp(-2*I*Kr)/2 + O_{2,1}*exp(2*I*Kr)/2 + O_{2,2}*exp(-I*\theta_{κ_1})/2 + exp(2*I*Kr)*conjugate(O_{1,2})/2 - exp(I*\theta_{κ_1})*conjugate(O_{1,1})/2 - exp(-I*\theta_{κ_1})*conjugate(O_{2,2})/2 + exp(-2*I*Kr)*conjugate(O_{2,1})/2],
[-O_{1,1}*exp(-I*\theta_{κ_1})/2 + O_{1,2}*exp(-2*I*Kr)/2 + O_{2,1}*exp(2*I*Kr)/2 - O_{2,2}*exp(I*\theta_{κ_1})/2 + exp(2*I*Kr)*conjugate(O_{1,2})/2 + exp(I*\theta_{κ_1})*conjugate(O_{2,2})/2 + exp(-I*\theta_{κ_1})*conjugate(O_{1,1})/2 + exp(-2*I*Kr)*conjugate(O_{2,1})/2, -O_{1,1}/2 - O_{1,2}*exp(-2*I*Kr)*exp(-I*\theta_{κ_1})/2 + O_{2,1}*exp(2*I*Kr)*exp(I*\theta_{κ_1})/2 + O_{2,2}/2 - exp(2*I*Kr)*exp(I*\theta_{κ_1})*conjugate(O_{1,2})/2 - co

In [236]:
S['y'].subs(rules_O).subs(rules_intra).subs(rules_Oab).expand().xreplace(rules_sum)

NameError: name 'rules_sum' is not defined

In [ ]:
S['x'].subs(rules_O).subs(rules_inter).subs(rules_Oab).expand()

In [ ]:
(matrix_rewrite(S['y'].subs(rules_O).subs(rules_inter).subs(rules_Oab).expand().xreplace(rules_sum),cos).expand())

In [ ]:
S['y'].subs(rules_O).subs(rules_inter).subs(rules_Oab).expand()

In [ ]:
matrix_rewrite(S['y'].subs(rules_O).subs(rules_inter).subs(rules_Oab).expand(),sin).expand()

In [95]:
def matrix_rewrite(mat,func,simplify=False):
    _shape=mat.shape
    if simplify:
        return Matrix([[mat[i,j].rewrite(func).simplify() for j in range(_shape[0])] for i in range(_shape[1])])
    else:
        return Matrix([[mat[i,j].rewrite(func) for j in range(_shape[0])] for i in range(_shape[1])])



## s-wave order parameter

Assume real order parameter 

In [117]:
rules_O_s=[(O[(i,j)],Re[(i,j)]) for i in range(1,3) for j in range(1,3)]

In [239]:
rules_O_s_im=[(O[(i,j)],I*Im[(i,j)]) for i in range(1,3) for j in range(1,3)]

In [131]:
rules_O_s_reim=[(O[(i,j)],Re[(i,j)]+I*Im[(i,j)]) for i in range(1,3) for j in range(1,3)]

In [136]:
rules_sum={exp(k*I*theta[1]):0 for k in range(-10,11)}

In [178]:
matrix_rewrite(S[('y',0)].subs(rules_O).subs(rules_inter).subs(rules_O_s).expand().xreplace(rules_sum),exp)

Matrix([
[                                                                                                      0, I*a_{1,2}*exp(2*I*Kr)/2 - I*a_{1,2}*exp(-2*I*Kr)/2 - I*a_{2,1}*exp(2*I*Kr)/2 + I*a_{2,1}*exp(-2*I*Kr)/2],
[I*a_{1,2}*exp(2*I*Kr)/2 - I*a_{1,2}*exp(-2*I*Kr)/2 - I*a_{2,1}*exp(2*I*Kr)/2 + I*a_{2,1}*exp(-2*I*Kr)/2,                                                                                                       0]])

In [183]:
matrix_rewrite(S[('y',2)].subs(rules_O).subs(rules_intra).subs(rules_O_s_reim).expand().xreplace(rules_sum),exp)

Matrix([
[b_{1,1}*exp(2*I*pi/3) - b_{2,2}*exp(-2*I*pi/3),                                               0],
[                                             0, -b_{1,1}*exp(2*I*pi/3) + b_{2,2}*exp(-2*I*pi/3)]])

In [139]:
matrix_rewrite(S[('x',2)].subs(rules_O).subs(rules_intra).subs(rules_O_s_reim).expand().xreplace(rules_sum),exp)

Matrix([
[a_{1,1}*exp(2*I*pi/3) - a_{2,2}*exp(-2*I*pi/3),                                               0],
[                                             0, -a_{1,1}*exp(2*I*pi/3) + a_{2,2}*exp(-2*I*pi/3)]])

In [341]:
S[('x',0)].subs(rules_O).subs(rules_inter).subs(rules_O_s).expand()

Matrix([
[-a_{1,2}*exp(2*I*Kr)*exp(-I*\theta_{κ_1})/2 - a_{1,2}*exp(-2*I*Kr)*exp(I*\theta_{κ_1})/2 + a_{2,1}*exp(2*I*Kr)*exp(-I*\theta_{κ_1})/2 + a_{2,1}*exp(-2*I*Kr)*exp(I*\theta_{κ_1})/2,                                                                                    a_{1,2}*exp(2*I*Kr)/2 + a_{1,2}*exp(-2*I*Kr)/2 + a_{2,1}*exp(2*I*Kr)/2 + a_{2,1}*exp(-2*I*Kr)/2],
[                                                                                   a_{1,2}*exp(2*I*Kr)/2 + a_{1,2}*exp(-2*I*Kr)/2 + a_{2,1}*exp(2*I*Kr)/2 + a_{2,1}*exp(-2*I*Kr)/2, -a_{1,2}*exp(2*I*Kr)*exp(I*\theta_{κ_1})/2 - a_{1,2}*exp(-2*I*Kr)*exp(-I*\theta_{κ_1})/2 + a_{2,1}*exp(2*I*Kr)*exp(I*\theta_{κ_1})/2 + a_{2,1}*exp(-2*I*Kr)*exp(-I*\theta_{κ_1})/2]])

In [340]:
S[('x',0)].subs(rules_O).subs(rules_inter).expand().xreplace(rules_sum)

Matrix([
[                                                                                                                    0, O_{1,2}*exp(-2*I*Kr)/2 + O_{2,1}*exp(2*I*Kr)/2 + exp(2*I*Kr)*conjugate(O_{1,2})/2 + exp(-2*I*Kr)*conjugate(O_{2,1})/2],
[O_{1,2}*exp(-2*I*Kr)/2 + O_{2,1}*exp(2*I*Kr)/2 + exp(2*I*Kr)*conjugate(O_{1,2})/2 + exp(-2*I*Kr)*conjugate(O_{2,1})/2,                                                                                                                     0]])

In [342]:
S[('x',1)].subs(rules_O).subs(rules_inter).expand().xreplace(rules_sum)

Matrix([
[                                                                                                                                                                          0, (-1)**(2/3)*O_{1,2}*exp(-2*I*Kr)/2 + O_{2,1}*exp(-2*I*pi/3)*exp(2*I*Kr)/2 + exp(-2*I*pi/3)*exp(2*I*Kr)*conjugate(O_{1,2})/2 + (-1)**(2/3)*exp(-2*I*Kr)*conjugate(O_{2,1})/2],
[(-1)**(2/3)*O_{1,2}*exp(-2*I*Kr)/2 + O_{2,1}*exp(-2*I*pi/3)*exp(2*I*Kr)/2 + exp(-2*I*pi/3)*exp(2*I*Kr)*conjugate(O_{1,2})/2 + (-1)**(2/3)*exp(-2*I*Kr)*conjugate(O_{2,1})/2,                                                                                                                                                                           0]])

In [344]:
S[('x',1)].subs(rules_O).subs(rules_inter).subs(rules_O_s).expand().xreplace(rules_sum)

Matrix([
[                                                                                                                                                    0, a_{1,2}*exp(-2*I*pi/3)*exp(2*I*Kr)/2 + (-1)**(2/3)*a_{1,2}*exp(-2*I*Kr)/2 + a_{2,1}*exp(-2*I*pi/3)*exp(2*I*Kr)/2 + (-1)**(2/3)*a_{2,1}*exp(-2*I*Kr)/2],
[a_{1,2}*exp(-2*I*pi/3)*exp(2*I*Kr)/2 + (-1)**(2/3)*a_{1,2}*exp(-2*I*Kr)/2 + a_{2,1}*exp(-2*I*pi/3)*exp(2*I*Kr)/2 + (-1)**(2/3)*a_{2,1}*exp(-2*I*Kr)/2,                                                                                                                                                     0]])

In [345]:
S[('x',2)].subs(rules_O).subs(rules_inter).subs(rules_O_s).expand().xreplace(rules_sum)

Matrix([
[                                                                                                                                                    0, (-1)**(2/3)*a_{1,2}*exp(2*I*Kr)/2 + a_{1,2}*exp(-2*I*pi/3)*exp(-2*I*Kr)/2 + (-1)**(2/3)*a_{2,1}*exp(2*I*Kr)/2 + a_{2,1}*exp(-2*I*pi/3)*exp(-2*I*Kr)/2],
[(-1)**(2/3)*a_{1,2}*exp(2*I*Kr)/2 + a_{1,2}*exp(-2*I*pi/3)*exp(-2*I*Kr)/2 + (-1)**(2/3)*a_{2,1}*exp(2*I*Kr)/2 + a_{2,1}*exp(-2*I*pi/3)*exp(-2*I*Kr)/2,                                                                                                                                                     0]])

In [244]:
S['y'].subs(rules_O).subs(rules_intra).subs(rules_O_s_im).expand().xreplace(rules_sum)

Matrix([
[b_{1,1} - b_{2,2},                  0],
[                0, -b_{1,1} + b_{2,2}]])

In [ ]:
matrix_rewrite(S['x'].subs(rules_O).subs(rules_inter).subs(rules_O_s).expand().xreplace(rules_sum),cos).expand()

In [ ]:
matrix_rewrite(S['x'].subs(rules_O).subs(rules_inter).subs(rules_O_s_im).expand().xreplace(rules_sum),cos).expand()

In [ ]:
matrix_rewrite(S['y'].subs(rules_O).subs(rules_inter).subs(rules_O_s).expand().xreplace(rules_sum),cos).expand()

In [ ]:
matrix_rewrite(S['y'].subs(rules_O).subs(rules_inter).subs(rules_O_s_im).expand().xreplace(rules_sum),cos).expand()

In [ ]:
COO.subs(rules_intra).subs(rules_O_s)

In [ ]:
H0=(Matrix([Dagger(c['+Au1']),Dagger(c['+Bu1'])]).T * (vF*(sigma['x']*cos(theta[1])+sigma['y']*sin(theta[1])) + B*  sigma['0']) * Matrix([c['+Au1'],c['+Bu1']]) \
+ Matrix([Dagger(c['-Au1']),Dagger(c['-Bu1'])]).T * conjugate(vF*(-sigma['x']*cos(theta[1])-sigma['y']*sin(theta[1])) + B*  sigma['0']) * Matrix([c['-Au1'],c['-Bu1']])\
+ Matrix([Dagger(c['+Ad1']),Dagger(c['+Bd1'])]).T * (vF*(sigma['x']*cos(theta[1])+sigma['y']*sin(theta[1])) - B*  sigma['0'] ) * Matrix([c['+Ad1'],c['+Bd1']])\
+Matrix([Dagger(c['-Ad1']),Dagger(c['-Bd1'])]).T * conjugate(vF*(-sigma['x']*cos(theta[1])-sigma['y']*sin(theta[1])) - B*  sigma['0']) * Matrix([c['-Ad1'],c['-Bd1']])\
)[0,0].rewrite(exp)

In [109]:
z8=zeros(8,8)
mat={}
for i in range(8):
    for j in range(8):
        tmp=z8.copy()
        tmp[i,j]=1
        mat[(i,j)]=tmp.copy()
        
rules_c2mat=list(zip(Matrix([c_d_vec[1][i]* c_vec[1][j] for i in range(8) for j in range(8)]),[mat[(i,j)] for i in range(8) for j in range(8)]))

In [ ]:
H_re=H0.expand().subs(rules_c2mat)+COO.subs(rules_intra).subs(rules_O_s).subs([(Re[2,2],Re[1,1])])

In [ ]:
H_re_={}
H_re_['+K']=H_re[[0,1,4,5],[0,1,4,5]]
H_re_['-K']=H_re[[2,3,6,7],[2,3,6,7]]

In [ ]:
H_re

In [ ]:
H_re_['+K']

In [ ]:
vec,val=(H_re_['+K']).diagonalize()

In [ ]:
matrix_rewrite(vec,cos).expand()

In [ ]:
exp(I*theta[1])*kronecker_product(sigma['+'],sigma['+'])-exp(-I*theta[1])*kronecker_product(sigma['+'],sigma['-'])-exp(I*theta[1])*kronecker_product(sigma['-'],sigma['+'])+exp(-I*theta[1])*kronecker_product(sigma['-'],sigma['-'])

In [ ]:
from sympy.physics.paulialgebra import Pauli, evaluate_pauli_product

In [ ]:
SS={i:Operator(f'S_{i}') for i in '+-xyz'}
Sigma={i:Operator(f'\Sigma_{i}') for i in '+-xyz'}

In [ ]:
rules_pauli=[(SS['+'],(SS['x'] + I* SS['y'])/2),(SS['-'],(SS['x'] - I* SS['y'])/2),(Sigma['+'],(Sigma['x'] + I* Sigma['y'])/2),(Sigma['-'],(Sigma['x'] - I* Sigma['y'])/2)]

In [ ]:
(exp(I*theta[1])*(Sigma['+']*SS['+'])-exp(-I*theta[1])*(Sigma['+']*SS['-'])-exp(I*theta[1])*(Sigma['-']*SS['+'])+exp(-I*theta[1])*(Sigma['-']*SS['-'])).subs(rules_pauli).rewrite(cos).expand()

In [ ]:
matrix_rewrite(val,cos,simplify=True)

## p-wave order parameter

In [156]:
r={(i,j): Symbol(f'r_{{{i},{j}}}') for i in range(1,3) for j in range(1,3)}

In [157]:
rules_O_p=[(O[(i,j)],r[(i,j)]*exp(I*theta[1])) for i in range(1,3) for j in range(1,3)]
rules_O_p_intra_opposite=[(O[(i,j)],r[(i,j)]*exp((-1)**i*I*theta[1])) for i,j in zip(range(1,3),range(1,3))]
rules_O_p_inter_opposite=[(O[(i,j)],r[(i,j)]*exp((-1)**i*I*theta[1])) for i,j in [(1,2),(2,1)]]

In [175]:
matrix_rewrite(S[('y',1)].subs(rules_O).subs(rules_intra).subs(rules_O_p).xreplace(rules_sum),exp)

Matrix([
[                                                            0, r_{2,2}*exp(I*pi/6)/2 - I*exp(-2*I*pi/3)*conjugate(r_{1,1})/2],
[I*r_{1,1}*exp(-2*I*pi/3)/2 - exp(I*pi/6)*conjugate(r_{2,2})/2,                                                             0]])

In [256]:
S['y'].subs(rules_O).subs(rules_intra).subs(rules_O_p).xreplace(rules_sum)

Matrix([
[                                   0, -I*r_{2,2}/2 - I*conjugate(r_{1,1})/2],
[I*r_{1,1}/2 + I*conjugate(r_{2,2})/2,                                     0]])

In [172]:
matrix_rewrite(S[('y',2)].subs(rules_O).subs(rules_inter).subs(rules_O_p).xreplace(rules_sum),exp)

Matrix([
[r_{2,1}*exp(I*pi/6)*exp(2*I*Kr)/2 + I*exp(-2*I*pi/3)*exp(-2*I*Kr)*conjugate(r_{2,1})/2,                                                                                      0],
[                                                                                     0, I*r_{1,2}*exp(-2*I*pi/3)*exp(-2*I*Kr)/2 + exp(I*pi/6)*exp(2*I*Kr)*conjugate(r_{1,2})/2]])

In [263]:
S['x'].subs(rules_O).subs(rules_intra).subs(rules_O_p_intra_opposite)

Matrix([
[r_{1,1}*exp(-I*\theta_{κ_1})/2 - r_{2,2}*exp(I*\theta_{κ_1})/2 + exp(I*\theta_{κ_1})*conjugate(r_{1,1})/2 - exp(-I*\theta_{κ_1})*conjugate(r_{2,2})/2,                                       r_{1,1}/2 + r_{2,2}/2 - exp(2*I*\theta_{κ_1})*conjugate(r_{1,1})/2 - exp(-2*I*\theta_{κ_1})*conjugate(r_{2,2})/2],
[                                    -r_{1,1}*exp(-2*I*\theta_{κ_1})/2 - r_{2,2}*exp(2*I*\theta_{κ_1})/2 + conjugate(r_{1,1})/2 + conjugate(r_{2,2})/2, -r_{1,1}*exp(-I*\theta_{κ_1})/2 + r_{2,2}*exp(I*\theta_{κ_1})/2 - exp(I*\theta_{κ_1})*conjugate(r_{1,1})/2 + exp(-I*\theta_{κ_1})*conjugate(r_{2,2})/2]])

In [264]:
S['y'].subs(rules_O).subs(rules_intra).subs(rules_O_p_intra_opposite).xreplace(rules_sum)

Matrix([
[                                              0, -I*r_{1,1}/2 - I*r_{2,2}/2],
[I*conjugate(r_{1,1})/2 + I*conjugate(r_{2,2})/2,                          0]])

In [267]:
S['x'].subs(rules_O).subs(rules_inter).subs(rules_O_p).xreplace(rules_sum)

Matrix([
[r_{2,1}*exp(2*I*Kr)/2 + exp(-2*I*Kr)*conjugate(r_{2,1})/2,                                                          0],
[                                                        0, -r_{1,2}*exp(-2*I*Kr)/2 - exp(2*I*Kr)*conjugate(r_{1,2})/2]])

In [268]:
S['y'].subs(rules_O).subs(rules_inter).subs(rules_O_p).xreplace(rules_sum)

Matrix([
[-I*r_{2,1}*exp(2*I*Kr)/2 + I*exp(-2*I*Kr)*conjugate(r_{2,1})/2,                                                             0],
[                                                             0, I*r_{1,2}*exp(-2*I*Kr)/2 - I*exp(2*I*Kr)*conjugate(r_{1,2})/2]])

In [269]:
S['x'].subs(rules_O).subs(rules_inter).subs(rules_O_p_inter_opposite).xreplace(rules_sum)

Matrix([
[-r_{1,2}*exp(-2*I*Kr)/2 + r_{2,1}*exp(2*I*Kr)/2 - exp(2*I*Kr)*conjugate(r_{1,2})/2 + exp(-2*I*Kr)*conjugate(r_{2,1})/2, 0],
[                                                                                                                     0, 0]])

In [270]:
S['y'].subs(rules_O).subs(rules_inter).subs(rules_O_p_inter_opposite).xreplace(rules_sum)

Matrix([
[I*r_{1,2}*exp(-2*I*Kr)/2 - I*r_{2,1}*exp(2*I*Kr)/2 - I*exp(2*I*Kr)*conjugate(r_{1,2})/2 + I*exp(-2*I*Kr)*conjugate(r_{2,1})/2, 0],
[                                                                                                                            0, 0]])

## d-wave order parameter

In [ ]:
rules_O_d=[(O[(i,j)],r[(i,j)]*exp(2*I*theta[1])) for i in range(1,3) for j in range(1,3)]

In [ ]:
S['x'].subs(rules_O).subs(rules_intra).subs(rules_O_d).xreplace(rules_sum)

In [ ]:
S['y'].subs(rules_O).subs(rules_intra).subs(rules_O_d).xreplace(rules_sum)

In [ ]:
S['x'].subs(rules_O).subs(rules_inter).subs(rules_O_d).xreplace(rules_sum)

In [ ]:
S['y'].subs(rules_O).subs(rules_inter).subs(rules_O_d).xreplace(rules_sum)

In [ ]:
WW[1]

In [ ]:
(psi['++d1']*theta[2]+psi['++d1']*theta[1]).factor()

In [ ]:
(theta[3]*theta[2]+theta[3]*theta[1])

# C3 symm

In [ ]:
C3=Operator(f'C_3')

In [ ]:
Matrix([c[f'+{l}u1'] for l in 'AB'])

In [ ]:
omega=Symbol(r'\omega')

In [ ]:
diag(1,omega, 1, omega**-1)

In [ ]:
(R(0)*K['+']).dot(n[1])*4*pi/sqrt(3)

In [ ]:
R= lambda x: Matrix([[cos(rad(x)),-sin(rad(x))],[sin(rad(x)),cos(rad(x))]])

In [ ]:
R(120)*K['+']

In [ ]:
K['+'].dot(-n[2])*4*pi/sqrt(3)

In [ ]:
n[1]